In [ ]:
##This script is fairly slow and now optimed paper formula is used instead.
##This takes 1h 40 min for 5 subsets while the optimed version takes 5 min

In [37]:
cd C:\Users\Yoo\Desktop\summerProject\hmmer-3.2.1\trial\usefulFiles\machineLearningKernelPfamFingerPrints\IKOR_TryingDifferentParameters\sameOrderForAllConfigurations\firstExperiment

C:\Users\Yoo\Desktop\summerProject\hmmer-3.2.1\trial\usefulFiles\machineLearningKernelPfamFingerPrints\IKOR_TryingDifferentParameters\sameOrderForAllConfigurations\firstExperiment


In [38]:
import numpy as np

In [39]:
def convertStringIntoVector (String):##Receives a binary vector in the form of a String
    intoList=list(String)
    vector=np.asarray(intoList,dtype=int)
    return vector

In [40]:
def gaussianKernelForPfam (vector1, vector2):##Does the gaussian Kernel for two vectors
    gamma=0.01
    euclideanDistance=np.linalg.norm(vector1-vector2)
    kernelValue=np.exp(-gamma*(euclideanDistance**2))
    return kernelValue

In [41]:
def gaussianKernelForFingerprint (vector1, vector2):##We can have a different gamma in this kernel if we want
    gamma=0.01
    euclideanDistance=np.linalg.norm(vector1-vector2)
    kernelValue=np.exp(-gamma*(euclideanDistance**2))
    return kernelValue

In [42]:
##Opens the two files and creates one list of Strings for pfam and another one for fingerprints
pfamFile= open ("pfamDomains0and1ForAllClusters.txt")
pfamList=[]##List of Strings
for line in pfamFile:
    line=line.strip("\n")
    pfamList.append(line[11:])
pfamFile.close()
fingerprintFile=open ("fingerPrintsForIndividualClustersNOT478.txt")
fingerprintList=[]##List of strings
for line in fingerprintFile:
    line=line.strip("\n")
    fingerprintList.append(line[11:])
fingerprintFile.close()

In [43]:
##Converts the Lists of strings into list of Vectors
pfamVectorList=[]#List of vectors
for pfamString in pfamList:
    pfamVectorList.append(convertStringIntoVector(pfamString))
fingerprintVectorList=[]##List of vectors
for fingerprintString in fingerprintList:
    fingerprintVectorList.append(convertStringIntoVector(fingerprintString))

In [44]:
##Sets parameters for the training part of the formula (lambda*Identity*GramMatrix) and inverses it
dimensionGeneralMatrix= len (pfamVectorList)
lambdaa=0.001 #lambda seems to be a special character in python so lets put a double aa
generalGramMatrix = np.zeros((dimensionGeneralMatrix,dimensionGeneralMatrix))
for i in range (0, dimensionGeneralMatrix):
    for j in range (0, dimensionGeneralMatrix):
        generalGramMatrix [i,j]=gaussianKernelForPfam(pfamVectorList[i],pfamVectorList[j])#Kx with upper case

In [45]:
import random
#Creates a list of Random Numbers as big as the number of Clusters we have
#Then we will select which ones we have for each subset
listOfRandomNumbers=random.sample(range(dimensionGeneralMatrix), dimensionGeneralMatrix)
sampleOf206Numbers=listOfRandomNumbers[0:206]
sampleOf825Numbers= listOfRandomNumbers[206:]

In [46]:
####This is just the same script repeated several times for the 5 subsets.
####The last time, the length of the training and the test set changes, but the rest is the same

##Creates the training part of the formula once you know the specific clusters in the training set

dimensionTrainingSet = 825 #We divide 1031 by 5 for crossvalidation. We have 825 in the training and 206 in the test
dimensionTestSet = 206
identityMatrix=np.identity(dimensionTrainingSet)##Part of the formula
specificGramMatrix = np.zeros((dimensionTrainingSet,dimensionTrainingSet))
for i in range (0, dimensionTrainingSet):
    for j in range (0, dimensionTrainingSet):
        iIndex=sampleOf825Numbers[i]
        jIndex=sampleOf825Numbers[j]
        specificGramMatrix[i,j]=generalGramMatrix[iIndex,jIndex]
inverseMatrix = np.linalg.inv( ( lambdaa * identityMatrix ) + specificGramMatrix )##Part of the formula

In [47]:
def calculatingIOKRScore (xVector, yVector):
    ky=np.zeros((dimensionTrainingSet,1))##kx lower case
    kx=np.zeros((dimensionTrainingSet,1))##ky lower case
    for i in range (0,dimensionTrainingSet):
        iIndex=sampleOf825Numbers[i]
        kx[i]=gaussianKernelForPfam(pfamVectorList[iIndex],xVector)
        ky[i]=gaussianKernelForFingerprint(fingerprintVectorList[iIndex],yVector)
    kyt=np.transpose(ky)##Transposing ky
    result= np.matmul(np.matmul(kyt,inverseMatrix), kx)##Returns a number when doing the multiplication of the matrixes
    return result

In [48]:
score = np.zeros((206, 206))
for i in range (0, 206):
    for j in range (0, 206):
        iIndex=sampleOf206Numbers[i]
        jIndex=sampleOf206Numbers[j]
        score[i,j]= calculatingIOKRScore(pfamVectorList[iIndex],fingerprintVectorList[jIndex])
np.save("score",score)

In [49]:
sampleOf206Numbers=listOfRandomNumbers[206:412]
sampleOf825Numbers= listOfRandomNumbers[0:206]+listOfRandomNumbers[412:]
##Creates the training part of the formula once you know the specific clusters in the training
dimensionTrainingSet = 825 #We divide 1031 by 5 for crossvalidation. We have 825 in the rest
dimensionTestSet = 206
identityMatrix=np.identity(dimensionTrainingSet)##Part of the formula
specificGramMatrix = np.zeros((dimensionTrainingSet,dimensionTrainingSet))
for i in range (0, dimensionTrainingSet):
    for j in range (0, dimensionTrainingSet):
        iIndex=sampleOf825Numbers[i]
        jIndex=sampleOf825Numbers[j]
        specificGramMatrix[i,j]=generalGramMatrix[iIndex,jIndex]
inverseMatrix = np.linalg.inv( ( lambdaa * identityMatrix ) + specificGramMatrix )##Part of the formula

In [50]:
def calculatingIOKRScore (xVector, yVector):
    ky=np.zeros((dimensionTrainingSet,1))##kx lower case
    kx=np.zeros((dimensionTrainingSet,1))##ky lower case
    for i in range (0,dimensionTrainingSet):
        iIndex=sampleOf825Numbers[i]
        kx[i]=gaussianKernelForPfam(pfamVectorList[iIndex],xVector)
        ky[i]=gaussianKernelForFingerprint(fingerprintVectorList[iIndex],yVector)
    kyt=np.transpose(ky)##Transposing ky
    result= np.matmul(np.matmul(kyt,inverseMatrix), kx)##Returns a number when doing the multiplication of the matrixes
    return result

In [51]:
score2 = np.zeros((206, 206))
for i in range (0, 206):
    for j in range (0, 206):
        iIndex=sampleOf206Numbers[i]
        jIndex=sampleOf206Numbers[j]
        score2[i,j]= calculatingIOKRScore(pfamVectorList[iIndex],fingerprintVectorList[jIndex])
np.save("score2",score2)

In [52]:
sampleOf206Numbers=listOfRandomNumbers[412:618]
sampleOf825Numbers= listOfRandomNumbers[0:412]+listOfRandomNumbers[618:]
##Creates the training part of the formula once you know the specific clusters in the training
dimensionTrainingSet = 825 #We divide 1031 by 5 for crossvalidation. We have 825 in the rest
dimensionTestSet = 206
identityMatrix=np.identity(dimensionTrainingSet)##Part of the formula
specificGramMatrix = np.zeros((dimensionTrainingSet,dimensionTrainingSet))
for i in range (0, dimensionTrainingSet):
    for j in range (0, dimensionTrainingSet):
        iIndex=sampleOf825Numbers[i]
        jIndex=sampleOf825Numbers[j]
        specificGramMatrix[i,j]=generalGramMatrix[iIndex,jIndex]
inverseMatrix = np.linalg.inv( ( lambdaa * identityMatrix ) + specificGramMatrix )##Part of the formula

In [53]:
def calculatingIOKRScore (xVector, yVector):
    ky=np.zeros((dimensionTrainingSet,1))##kx lower case
    kx=np.zeros((dimensionTrainingSet,1))##ky lower case
    for i in range (0,dimensionTrainingSet):
        iIndex=sampleOf825Numbers[i]
        kx[i]=gaussianKernelForPfam(pfamVectorList[iIndex],xVector)
        ky[i]=gaussianKernelForFingerprint(fingerprintVectorList[iIndex],yVector)
    kyt=np.transpose(ky)##Transposing ky
    result= np.matmul(np.matmul(kyt,inverseMatrix), kx)##Returns a number when doing the multiplication of the matrixes
    return result

In [54]:
score3 = np.zeros((206, 206))
for i in range (0, 206):
    for j in range (0, 206):
        iIndex=sampleOf206Numbers[i]
        jIndex=sampleOf206Numbers[j]
        score3[i,j]= calculatingIOKRScore(pfamVectorList[iIndex],fingerprintVectorList[jIndex])
np.save("score3",score3)

In [55]:
sampleOf206Numbers=listOfRandomNumbers[618:824]
sampleOf825Numbers= listOfRandomNumbers[0:618]+listOfRandomNumbers[824:]
##Creates the training part of the formula once you know the specific clusters in the training
dimensionTrainingSet = 825 #We divide 1031 by 5 for crossvalidation. We have 825 in the rest
dimensionTestSet = 206
identityMatrix=np.identity(dimensionTrainingSet)##Part of the formula
specificGramMatrix = np.zeros((dimensionTrainingSet,dimensionTrainingSet))
for i in range (0, dimensionTrainingSet):
    for j in range (0, dimensionTrainingSet):
        iIndex=sampleOf825Numbers[i]
        jIndex=sampleOf825Numbers[j]
        specificGramMatrix[i,j]=generalGramMatrix[iIndex,jIndex]
inverseMatrix = np.linalg.inv( ( lambdaa * identityMatrix ) + specificGramMatrix )##Part of the formula

In [56]:
def calculatingIOKRScore (xVector, yVector):
    ky=np.zeros((dimensionTrainingSet,1))##kx lower case
    kx=np.zeros((dimensionTrainingSet,1))##ky lower case
    for i in range (0,dimensionTrainingSet):
        iIndex=sampleOf825Numbers[i]
        kx[i]=gaussianKernelForPfam(pfamVectorList[iIndex],xVector)
        ky[i]=gaussianKernelForFingerprint(fingerprintVectorList[iIndex],yVector)
    kyt=np.transpose(ky)##Transposing ky
    result= np.matmul(np.matmul(kyt,inverseMatrix), kx)##Returns a number when doing the multiplication of the matrixes
    return result

In [57]:
score4 = np.zeros((206, 206))
for i in range (0, 206):
    for j in range (0, 206):
        iIndex=sampleOf206Numbers[i]
        jIndex=sampleOf206Numbers[j]
        score4[i,j]= calculatingIOKRScore(pfamVectorList[iIndex],fingerprintVectorList[jIndex])
np.save("score4",score4)

In [58]:
sampleOf206Numbers=listOfRandomNumbers[824:]
sampleOf825Numbers= listOfRandomNumbers[0:824]
##Creates the training part of the formula once you know the specific clusters in the training
dimensionTrainingSet = 824 #We divide 1031 by 5 for crossvalidation. We have 825 in the rest
dimensionTestSet = 207
identityMatrix=np.identity(dimensionTrainingSet)##Part of the formula
specificGramMatrix = np.zeros((dimensionTrainingSet,dimensionTrainingSet))
for i in range (0, dimensionTrainingSet):
    for j in range (0, dimensionTrainingSet):
        iIndex=sampleOf825Numbers[i]
        jIndex=sampleOf825Numbers[j]
        specificGramMatrix[i,j]=generalGramMatrix[iIndex,jIndex]
inverseMatrix = np.linalg.inv( ( lambdaa * identityMatrix ) + specificGramMatrix )##Part of the formula

In [59]:
def calculatingIOKRScore (xVector, yVector):
    ky=np.zeros((dimensionTrainingSet,1))##kx lower case
    kx=np.zeros((dimensionTrainingSet,1))##ky lower case
    for i in range (0,dimensionTrainingSet):
        iIndex=sampleOf825Numbers[i]
        kx[i]=gaussianKernelForPfam(pfamVectorList[iIndex],xVector)
        ky[i]=gaussianKernelForFingerprint(fingerprintVectorList[iIndex],yVector)
    kyt=np.transpose(ky)##Transposing ky
    result= np.matmul(np.matmul(kyt,inverseMatrix), kx)##Returns a number when doing the multiplication of the matrixes
    return result

In [60]:
score5 = np.zeros((207, 207))
for i in range (0, 207):
    for j in range (0, 207):
        iIndex=sampleOf206Numbers[i]
        jIndex=sampleOf206Numbers[j]
        score5[i,j]= calculatingIOKRScore(pfamVectorList[iIndex],fingerprintVectorList[jIndex])
np.save("score5",score5)

In [61]:
####The cell with import random has been deleted as we want to use the same order for all the parameters
## GammaX = 0.01
## GammaY = 0.01
## Lambda = 0.01

In [62]:
cd C:\Users\Yoo\Desktop\summerProject\hmmer-3.2.1\trial\usefulFiles\machineLearningKernelPfamFingerPrints\IKOR_TryingDifferentParameters\sameOrderForAllConfigurations\firstExperiment

C:\Users\Yoo\Desktop\summerProject\hmmer-3.2.1\trial\usefulFiles\machineLearningKernelPfamFingerPrints\IKOR_TryingDifferentParameters\sameOrderForAllConfigurations\firstExperiment


In [63]:
import numpy as np

In [64]:
def convertStringIntoVector (String):##Receives a binary vector in the form of a String
    intoList=list(String)
    vector=np.asarray(intoList,dtype=int)
    return vector

In [65]:
def gaussianKernelForPfam (vector1, vector2):##Does the gaussian Kernel for two vectors
    gamma=0.01
    euclideanDistance=np.linalg.norm(vector1-vector2)
    kernelValue=np.exp(-gamma*(euclideanDistance**2))
    return kernelValue

In [66]:
def gaussianKernelForFingerprint (vector1, vector2):##We can have a different gamma in this kernel if we want
    gamma=0.01
    euclideanDistance=np.linalg.norm(vector1-vector2)
    kernelValue=np.exp(-gamma*(euclideanDistance**2))
    return kernelValue

In [67]:
##Opens the two files and creates one list of Strings for pfam and another one for fingerprints
pfamFile= open ("pfamDomains0and1ForAllClusters.txt")
pfamList=[]##List of Strings
for line in pfamFile:
    line=line.strip("\n")
    pfamList.append(line[11:])
pfamFile.close()
fingerprintFile=open ("fingerPrintsForIndividualClustersNOT478.txt")
fingerprintList=[]##List of strings
for line in fingerprintFile:
    line=line.strip("\n")
    fingerprintList.append(line[11:])
fingerprintFile.close()

In [68]:
##Converts the Lists of strings into list of Vectors
pfamVectorList=[]#List of vectors
for pfamString in pfamList:
    pfamVectorList.append(convertStringIntoVector(pfamString))
fingerprintVectorList=[]##List of vectors
for fingerprintString in fingerprintList:
    fingerprintVectorList.append(convertStringIntoVector(fingerprintString))

In [69]:
##Sets parameters for the training part of the formula (lambda*Identity*GramMatrix) and inverses it
dimensionGeneralMatrix= len (pfamVectorList)
lambdaa=0.01 #lambda seems to be a special character in python so lets put a double aa
generalGramMatrix = np.zeros((dimensionGeneralMatrix,dimensionGeneralMatrix))
for i in range (0, dimensionGeneralMatrix):
    for j in range (0, dimensionGeneralMatrix):
        generalGramMatrix [i,j]=gaussianKernelForPfam(pfamVectorList[i],pfamVectorList[j])#Kx with upper case

In [70]:
sampleOf206Numbers=listOfRandomNumbers[0:206]
sampleOf825Numbers= listOfRandomNumbers[206:]
####This is just the same script repeated several times for the 5 subsets.
####The last time, the length of the training and the test set changes, but the rest is the same

##Creates the training part of the formula once you know the specific clusters in the training set

dimensionTrainingSet = 825 #We divide 1031 by 5 for crossvalidation. We have 825 in the training and 206 in the test
dimensionTestSet = 206
identityMatrix=np.identity(dimensionTrainingSet)##Part of the formula
specificGramMatrix = np.zeros((dimensionTrainingSet,dimensionTrainingSet))
for i in range (0, dimensionTrainingSet):
    for j in range (0, dimensionTrainingSet):
        iIndex=sampleOf825Numbers[i]
        jIndex=sampleOf825Numbers[j]
        specificGramMatrix[i,j]=generalGramMatrix[iIndex,jIndex]
inverseMatrix = np.linalg.inv( ( lambdaa * identityMatrix ) + specificGramMatrix )##Part of the formula

In [71]:
def calculatingIOKRScore (xVector, yVector):
    ky=np.zeros((dimensionTrainingSet,1))##kx lower case
    kx=np.zeros((dimensionTrainingSet,1))##ky lower case
    for i in range (0,dimensionTrainingSet):
        iIndex=sampleOf825Numbers[i]
        kx[i]=gaussianKernelForPfam(pfamVectorList[iIndex],xVector)
        ky[i]=gaussianKernelForFingerprint(fingerprintVectorList[iIndex],yVector)
    kyt=np.transpose(ky)##Transposing ky
    result= np.matmul(np.matmul(kyt,inverseMatrix), kx)##Returns a number when doing the multiplication of the matrixes
    return result

In [72]:
score = np.zeros((206, 206))
for i in range (0, 206):
    for j in range (0, 206):
        iIndex=sampleOf206Numbers[i]
        jIndex=sampleOf206Numbers[j]
        score[i,j]= calculatingIOKRScore(pfamVectorList[iIndex],fingerprintVectorList[jIndex])
np.save("score6",score)

In [73]:
sampleOf206Numbers=listOfRandomNumbers[206:412]
sampleOf825Numbers= listOfRandomNumbers[0:206]+listOfRandomNumbers[412:]
##Creates the training part of the formula once you know the specific clusters in the training
dimensionTrainingSet = 825 #We divide 1031 by 5 for crossvalidation. We have 825 in the rest
dimensionTestSet = 206
identityMatrix=np.identity(dimensionTrainingSet)##Part of the formula
specificGramMatrix = np.zeros((dimensionTrainingSet,dimensionTrainingSet))
for i in range (0, dimensionTrainingSet):
    for j in range (0, dimensionTrainingSet):
        iIndex=sampleOf825Numbers[i]
        jIndex=sampleOf825Numbers[j]
        specificGramMatrix[i,j]=generalGramMatrix[iIndex,jIndex]
inverseMatrix = np.linalg.inv( ( lambdaa * identityMatrix ) + specificGramMatrix )##Part of the formula

In [74]:
def calculatingIOKRScore (xVector, yVector):
    ky=np.zeros((dimensionTrainingSet,1))##kx lower case
    kx=np.zeros((dimensionTrainingSet,1))##ky lower case
    for i in range (0,dimensionTrainingSet):
        iIndex=sampleOf825Numbers[i]
        kx[i]=gaussianKernelForPfam(pfamVectorList[iIndex],xVector)
        ky[i]=gaussianKernelForFingerprint(fingerprintVectorList[iIndex],yVector)
    kyt=np.transpose(ky)##Transposing ky
    result= np.matmul(np.matmul(kyt,inverseMatrix), kx)##Returns a number when doing the multiplication of the matrixes
    return result

In [75]:
score2 = np.zeros((206, 206))
for i in range (0, 206):
    for j in range (0, 206):
        iIndex=sampleOf206Numbers[i]
        jIndex=sampleOf206Numbers[j]
        score2[i,j]= calculatingIOKRScore(pfamVectorList[iIndex],fingerprintVectorList[jIndex])
np.save("score7",score2)

In [76]:
sampleOf206Numbers=listOfRandomNumbers[412:618]
sampleOf825Numbers= listOfRandomNumbers[0:412]+listOfRandomNumbers[618:]
##Creates the training part of the formula once you know the specific clusters in the training
dimensionTrainingSet = 825 #We divide 1031 by 5 for crossvalidation. We have 825 in the rest
dimensionTestSet = 206
identityMatrix=np.identity(dimensionTrainingSet)##Part of the formula
specificGramMatrix = np.zeros((dimensionTrainingSet,dimensionTrainingSet))
for i in range (0, dimensionTrainingSet):
    for j in range (0, dimensionTrainingSet):
        iIndex=sampleOf825Numbers[i]
        jIndex=sampleOf825Numbers[j]
        specificGramMatrix[i,j]=generalGramMatrix[iIndex,jIndex]
inverseMatrix = np.linalg.inv( ( lambdaa * identityMatrix ) + specificGramMatrix )##Part of the formula

In [77]:
def calculatingIOKRScore (xVector, yVector):
    ky=np.zeros((dimensionTrainingSet,1))##kx lower case
    kx=np.zeros((dimensionTrainingSet,1))##ky lower case
    for i in range (0,dimensionTrainingSet):
        iIndex=sampleOf825Numbers[i]
        kx[i]=gaussianKernelForPfam(pfamVectorList[iIndex],xVector)
        ky[i]=gaussianKernelForFingerprint(fingerprintVectorList[iIndex],yVector)
    kyt=np.transpose(ky)##Transposing ky
    result= np.matmul(np.matmul(kyt,inverseMatrix), kx)##Returns a number when doing the multiplication of the matrixes
    return result

In [78]:
score3 = np.zeros((206, 206))
for i in range (0, 206):
    for j in range (0, 206):
        iIndex=sampleOf206Numbers[i]
        jIndex=sampleOf206Numbers[j]
        score3[i,j]= calculatingIOKRScore(pfamVectorList[iIndex],fingerprintVectorList[jIndex])
np.save("score8",score3)

In [79]:
sampleOf206Numbers=listOfRandomNumbers[618:824]
sampleOf825Numbers= listOfRandomNumbers[0:618]+listOfRandomNumbers[824:]
##Creates the training part of the formula once you know the specific clusters in the training
dimensionTrainingSet = 825 #We divide 1031 by 5 for crossvalidation. We have 825 in the rest
dimensionTestSet = 206
identityMatrix=np.identity(dimensionTrainingSet)##Part of the formula
specificGramMatrix = np.zeros((dimensionTrainingSet,dimensionTrainingSet))
for i in range (0, dimensionTrainingSet):
    for j in range (0, dimensionTrainingSet):
        iIndex=sampleOf825Numbers[i]
        jIndex=sampleOf825Numbers[j]
        specificGramMatrix[i,j]=generalGramMatrix[iIndex,jIndex]
inverseMatrix = np.linalg.inv( ( lambdaa * identityMatrix ) + specificGramMatrix )##Part of the formula

In [80]:
def calculatingIOKRScore (xVector, yVector):
    ky=np.zeros((dimensionTrainingSet,1))##kx lower case
    kx=np.zeros((dimensionTrainingSet,1))##ky lower case
    for i in range (0,dimensionTrainingSet):
        iIndex=sampleOf825Numbers[i]
        kx[i]=gaussianKernelForPfam(pfamVectorList[iIndex],xVector)
        ky[i]=gaussianKernelForFingerprint(fingerprintVectorList[iIndex],yVector)
    kyt=np.transpose(ky)##Transposing ky
    result= np.matmul(np.matmul(kyt,inverseMatrix), kx)##Returns a number when doing the multiplication of the matrixes
    return result

In [81]:
score4 = np.zeros((206, 206))
for i in range (0, 206):
    for j in range (0, 206):
        iIndex=sampleOf206Numbers[i]
        jIndex=sampleOf206Numbers[j]
        score4[i,j]= calculatingIOKRScore(pfamVectorList[iIndex],fingerprintVectorList[jIndex])
np.save("score9",score4)

In [82]:
sampleOf206Numbers=listOfRandomNumbers[824:]
sampleOf825Numbers= listOfRandomNumbers[0:824]
##Creates the training part of the formula once you know the specific clusters in the training
dimensionTrainingSet = 824 #We divide 1031 by 5 for crossvalidation. We have 825 in the rest
dimensionTestSet = 207
identityMatrix=np.identity(dimensionTrainingSet)##Part of the formula
specificGramMatrix = np.zeros((dimensionTrainingSet,dimensionTrainingSet))
for i in range (0, dimensionTrainingSet):
    for j in range (0, dimensionTrainingSet):
        iIndex=sampleOf825Numbers[i]
        jIndex=sampleOf825Numbers[j]
        specificGramMatrix[i,j]=generalGramMatrix[iIndex,jIndex]
inverseMatrix = np.linalg.inv( ( lambdaa * identityMatrix ) + specificGramMatrix )##Part of the formula

In [83]:
def calculatingIOKRScore (xVector, yVector):
    ky=np.zeros((dimensionTrainingSet,1))##kx lower case
    kx=np.zeros((dimensionTrainingSet,1))##ky lower case
    for i in range (0,dimensionTrainingSet):
        iIndex=sampleOf825Numbers[i]
        kx[i]=gaussianKernelForPfam(pfamVectorList[iIndex],xVector)
        ky[i]=gaussianKernelForFingerprint(fingerprintVectorList[iIndex],yVector)
    kyt=np.transpose(ky)##Transposing ky
    result= np.matmul(np.matmul(kyt,inverseMatrix), kx)##Returns a number when doing the multiplication of the matrixes
    return result

In [84]:
score5 = np.zeros((207, 207))
for i in range (0, 207):
    for j in range (0, 207):
        iIndex=sampleOf206Numbers[i]
        jIndex=sampleOf206Numbers[j]
        score5[i,j]= calculatingIOKRScore(pfamVectorList[iIndex],fingerprintVectorList[jIndex])
np.save("score10",score5)

In [85]:
####The cell with import random has been deleted as we want to use the same order for all the parameters
## GammaX = 0.01
## GammaY = 0.01
## Lambda = 0.1

In [86]:
cd C:\Users\Yoo\Desktop\summerProject\hmmer-3.2.1\trial\usefulFiles\machineLearningKernelPfamFingerPrints\IKOR_TryingDifferentParameters\sameOrderForAllConfigurations\firstExperiment

C:\Users\Yoo\Desktop\summerProject\hmmer-3.2.1\trial\usefulFiles\machineLearningKernelPfamFingerPrints\IKOR_TryingDifferentParameters\sameOrderForAllConfigurations\firstExperiment


In [87]:
import numpy as np

In [88]:
def convertStringIntoVector (String):##Receives a binary vector in the form of a String
    intoList=list(String)
    vector=np.asarray(intoList,dtype=int)
    return vector

In [89]:
def gaussianKernelForPfam (vector1, vector2):##Does the gaussian Kernel for two vectors
    gamma=0.01
    euclideanDistance=np.linalg.norm(vector1-vector2)
    kernelValue=np.exp(-gamma*(euclideanDistance**2))
    return kernelValue

In [90]:
def gaussianKernelForFingerprint (vector1, vector2):##We can have a different gamma in this kernel if we want
    gamma=0.01
    euclideanDistance=np.linalg.norm(vector1-vector2)
    kernelValue=np.exp(-gamma*(euclideanDistance**2))
    return kernelValue

In [91]:
##Opens the two files and creates one list of Strings for pfam and another one for fingerprints
pfamFile= open ("pfamDomains0and1ForAllClusters.txt")
pfamList=[]##List of Strings
for line in pfamFile:
    line=line.strip("\n")
    pfamList.append(line[11:])
pfamFile.close()
fingerprintFile=open ("fingerPrintsForIndividualClustersNOT478.txt")
fingerprintList=[]##List of strings
for line in fingerprintFile:
    line=line.strip("\n")
    fingerprintList.append(line[11:])
fingerprintFile.close()

In [92]:
##Converts the Lists of strings into list of Vectors
pfamVectorList=[]#List of vectors
for pfamString in pfamList:
    pfamVectorList.append(convertStringIntoVector(pfamString))
fingerprintVectorList=[]##List of vectors
for fingerprintString in fingerprintList:
    fingerprintVectorList.append(convertStringIntoVector(fingerprintString))

In [93]:
##Sets parameters for the training part of the formula (lambda*Identity*GramMatrix) and inverses it
dimensionGeneralMatrix= len (pfamVectorList)
lambdaa=0.1 #lambda seems to be a special character in python so lets put a double aa
generalGramMatrix = np.zeros((dimensionGeneralMatrix,dimensionGeneralMatrix))
for i in range (0, dimensionGeneralMatrix):
    for j in range (0, dimensionGeneralMatrix):
        generalGramMatrix [i,j]=gaussianKernelForPfam(pfamVectorList[i],pfamVectorList[j])#Kx with upper case

In [94]:
sampleOf206Numbers=listOfRandomNumbers[0:206]
sampleOf825Numbers= listOfRandomNumbers[206:]
####This is just the same script repeated several times for the 5 subsets.
####The last time, the length of the training and the test set changes, but the rest is the same

##Creates the training part of the formula once you know the specific clusters in the training set

dimensionTrainingSet = 825 #We divide 1031 by 5 for crossvalidation. We have 825 in the training and 206 in the test
dimensionTestSet = 206
identityMatrix=np.identity(dimensionTrainingSet)##Part of the formula
specificGramMatrix = np.zeros((dimensionTrainingSet,dimensionTrainingSet))
for i in range (0, dimensionTrainingSet):
    for j in range (0, dimensionTrainingSet):
        iIndex=sampleOf825Numbers[i]
        jIndex=sampleOf825Numbers[j]
        specificGramMatrix[i,j]=generalGramMatrix[iIndex,jIndex]
inverseMatrix = np.linalg.inv( ( lambdaa * identityMatrix ) + specificGramMatrix )##Part of the formula

In [95]:
def calculatingIOKRScore (xVector, yVector):
    ky=np.zeros((dimensionTrainingSet,1))##kx lower case
    kx=np.zeros((dimensionTrainingSet,1))##ky lower case
    for i in range (0,dimensionTrainingSet):
        iIndex=sampleOf825Numbers[i]
        kx[i]=gaussianKernelForPfam(pfamVectorList[iIndex],xVector)
        ky[i]=gaussianKernelForFingerprint(fingerprintVectorList[iIndex],yVector)
    kyt=np.transpose(ky)##Transposing ky
    result= np.matmul(np.matmul(kyt,inverseMatrix), kx)##Returns a number when doing the multiplication of the matrixes
    return result

In [96]:
score = np.zeros((206, 206))
for i in range (0, 206):
    for j in range (0, 206):
        iIndex=sampleOf206Numbers[i]
        jIndex=sampleOf206Numbers[j]
        score[i,j]= calculatingIOKRScore(pfamVectorList[iIndex],fingerprintVectorList[jIndex])
np.save("score11",score)

In [97]:
sampleOf206Numbers=listOfRandomNumbers[206:412]
sampleOf825Numbers= listOfRandomNumbers[0:206]+listOfRandomNumbers[412:]
##Creates the training part of the formula once you know the specific clusters in the training
dimensionTrainingSet = 825 #We divide 1031 by 5 for crossvalidation. We have 825 in the rest
dimensionTestSet = 206
identityMatrix=np.identity(dimensionTrainingSet)##Part of the formula
specificGramMatrix = np.zeros((dimensionTrainingSet,dimensionTrainingSet))
for i in range (0, dimensionTrainingSet):
    for j in range (0, dimensionTrainingSet):
        iIndex=sampleOf825Numbers[i]
        jIndex=sampleOf825Numbers[j]
        specificGramMatrix[i,j]=generalGramMatrix[iIndex,jIndex]
inverseMatrix = np.linalg.inv( ( lambdaa * identityMatrix ) + specificGramMatrix )##Part of the formula

In [98]:
def calculatingIOKRScore (xVector, yVector):
    ky=np.zeros((dimensionTrainingSet,1))##kx lower case
    kx=np.zeros((dimensionTrainingSet,1))##ky lower case
    for i in range (0,dimensionTrainingSet):
        iIndex=sampleOf825Numbers[i]
        kx[i]=gaussianKernelForPfam(pfamVectorList[iIndex],xVector)
        ky[i]=gaussianKernelForFingerprint(fingerprintVectorList[iIndex],yVector)
    kyt=np.transpose(ky)##Transposing ky
    result= np.matmul(np.matmul(kyt,inverseMatrix), kx)##Returns a number when doing the multiplication of the matrixes
    return result

In [99]:
score2 = np.zeros((206, 206))
for i in range (0, 206):
    for j in range (0, 206):
        iIndex=sampleOf206Numbers[i]
        jIndex=sampleOf206Numbers[j]
        score2[i,j]= calculatingIOKRScore(pfamVectorList[iIndex],fingerprintVectorList[jIndex])
np.save("score12",score2)

In [100]:
sampleOf206Numbers=listOfRandomNumbers[412:618]
sampleOf825Numbers= listOfRandomNumbers[0:412]+listOfRandomNumbers[618:]
##Creates the training part of the formula once you know the specific clusters in the training
dimensionTrainingSet = 825 #We divide 1031 by 5 for crossvalidation. We have 825 in the rest
dimensionTestSet = 206
identityMatrix=np.identity(dimensionTrainingSet)##Part of the formula
specificGramMatrix = np.zeros((dimensionTrainingSet,dimensionTrainingSet))
for i in range (0, dimensionTrainingSet):
    for j in range (0, dimensionTrainingSet):
        iIndex=sampleOf825Numbers[i]
        jIndex=sampleOf825Numbers[j]
        specificGramMatrix[i,j]=generalGramMatrix[iIndex,jIndex]
inverseMatrix = np.linalg.inv( ( lambdaa * identityMatrix ) + specificGramMatrix )##Part of the formula

In [101]:
def calculatingIOKRScore (xVector, yVector):
    ky=np.zeros((dimensionTrainingSet,1))##kx lower case
    kx=np.zeros((dimensionTrainingSet,1))##ky lower case
    for i in range (0,dimensionTrainingSet):
        iIndex=sampleOf825Numbers[i]
        kx[i]=gaussianKernelForPfam(pfamVectorList[iIndex],xVector)
        ky[i]=gaussianKernelForFingerprint(fingerprintVectorList[iIndex],yVector)
    kyt=np.transpose(ky)##Transposing ky
    result= np.matmul(np.matmul(kyt,inverseMatrix), kx)##Returns a number when doing the multiplication of the matrixes
    return result

In [102]:
score3 = np.zeros((206, 206))
for i in range (0, 206):
    for j in range (0, 206):
        iIndex=sampleOf206Numbers[i]
        jIndex=sampleOf206Numbers[j]
        score3[i,j]= calculatingIOKRScore(pfamVectorList[iIndex],fingerprintVectorList[jIndex])
np.save("score13",score3)

In [103]:
sampleOf206Numbers=listOfRandomNumbers[618:824]
sampleOf825Numbers= listOfRandomNumbers[0:618]+listOfRandomNumbers[824:]
##Creates the training part of the formula once you know the specific clusters in the training
dimensionTrainingSet = 825 #We divide 1031 by 5 for crossvalidation. We have 825 in the rest
dimensionTestSet = 206
identityMatrix=np.identity(dimensionTrainingSet)##Part of the formula
specificGramMatrix = np.zeros((dimensionTrainingSet,dimensionTrainingSet))
for i in range (0, dimensionTrainingSet):
    for j in range (0, dimensionTrainingSet):
        iIndex=sampleOf825Numbers[i]
        jIndex=sampleOf825Numbers[j]
        specificGramMatrix[i,j]=generalGramMatrix[iIndex,jIndex]
inverseMatrix = np.linalg.inv( ( lambdaa * identityMatrix ) + specificGramMatrix )##Part of the formula

In [104]:
def calculatingIOKRScore (xVector, yVector):
    ky=np.zeros((dimensionTrainingSet,1))##kx lower case
    kx=np.zeros((dimensionTrainingSet,1))##ky lower case
    for i in range (0,dimensionTrainingSet):
        iIndex=sampleOf825Numbers[i]
        kx[i]=gaussianKernelForPfam(pfamVectorList[iIndex],xVector)
        ky[i]=gaussianKernelForFingerprint(fingerprintVectorList[iIndex],yVector)
    kyt=np.transpose(ky)##Transposing ky
    result= np.matmul(np.matmul(kyt,inverseMatrix), kx)##Returns a number when doing the multiplication of the matrixes
    return result

In [105]:
score4 = np.zeros((206, 206))
for i in range (0, 206):
    for j in range (0, 206):
        iIndex=sampleOf206Numbers[i]
        jIndex=sampleOf206Numbers[j]
        score4[i,j]= calculatingIOKRScore(pfamVectorList[iIndex],fingerprintVectorList[jIndex])
np.save("score14",score4)

In [106]:
sampleOf206Numbers=listOfRandomNumbers[824:]
sampleOf825Numbers= listOfRandomNumbers[0:824]
##Creates the training part of the formula once you know the specific clusters in the training
dimensionTrainingSet = 824 #We divide 1031 by 5 for crossvalidation. We have 825 in the rest
dimensionTestSet = 207
identityMatrix=np.identity(dimensionTrainingSet)##Part of the formula
specificGramMatrix = np.zeros((dimensionTrainingSet,dimensionTrainingSet))
for i in range (0, dimensionTrainingSet):
    for j in range (0, dimensionTrainingSet):
        iIndex=sampleOf825Numbers[i]
        jIndex=sampleOf825Numbers[j]
        specificGramMatrix[i,j]=generalGramMatrix[iIndex,jIndex]
inverseMatrix = np.linalg.inv( ( lambdaa * identityMatrix ) + specificGramMatrix )##Part of the formula

In [107]:
def calculatingIOKRScore (xVector, yVector):
    ky=np.zeros((dimensionTrainingSet,1))##kx lower case
    kx=np.zeros((dimensionTrainingSet,1))##ky lower case
    for i in range (0,dimensionTrainingSet):
        iIndex=sampleOf825Numbers[i]
        kx[i]=gaussianKernelForPfam(pfamVectorList[iIndex],xVector)
        ky[i]=gaussianKernelForFingerprint(fingerprintVectorList[iIndex],yVector)
    kyt=np.transpose(ky)##Transposing ky
    result= np.matmul(np.matmul(kyt,inverseMatrix), kx)##Returns a number when doing the multiplication of the matrixes
    return result

In [108]:
score5 = np.zeros((207, 207))
for i in range (0, 207):
    for j in range (0, 207):
        iIndex=sampleOf206Numbers[i]
        jIndex=sampleOf206Numbers[j]
        score5[i,j]= calculatingIOKRScore(pfamVectorList[iIndex],fingerprintVectorList[jIndex])
np.save("score15",score5)

In [109]:
####The cell with import random has been deleted as we want to use the same order for all the parameters
## GammaX = 0.003
## GammaY = 0.01
## Lambda = 0.01

In [110]:
cd C:\Users\Yoo\Desktop\summerProject\hmmer-3.2.1\trial\usefulFiles\machineLearningKernelPfamFingerPrints\IKOR_TryingDifferentParameters\sameOrderForAllConfigurations\firstExperiment

C:\Users\Yoo\Desktop\summerProject\hmmer-3.2.1\trial\usefulFiles\machineLearningKernelPfamFingerPrints\IKOR_TryingDifferentParameters\sameOrderForAllConfigurations\firstExperiment


In [111]:
import numpy as np

In [112]:
def convertStringIntoVector (String):##Receives a binary vector in the form of a String
    intoList=list(String)
    vector=np.asarray(intoList,dtype=int)
    return vector

In [113]:
def gaussianKernelForPfam (vector1, vector2):##Does the gaussian Kernel for two vectors
    gamma=0.003
    euclideanDistance=np.linalg.norm(vector1-vector2)
    kernelValue=np.exp(-gamma*(euclideanDistance**2))
    return kernelValue

In [114]:
def gaussianKernelForFingerprint (vector1, vector2):##We can have a different gamma in this kernel if we want
    gamma=0.01
    euclideanDistance=np.linalg.norm(vector1-vector2)
    kernelValue=np.exp(-gamma*(euclideanDistance**2))
    return kernelValue

In [115]:
##Opens the two files and creates one list of Strings for pfam and another one for fingerprints
pfamFile= open ("pfamDomains0and1ForAllClusters.txt")
pfamList=[]##List of Strings
for line in pfamFile:
    line=line.strip("\n")
    pfamList.append(line[11:])
pfamFile.close()
fingerprintFile=open ("fingerPrintsForIndividualClustersNOT478.txt")
fingerprintList=[]##List of strings
for line in fingerprintFile:
    line=line.strip("\n")
    fingerprintList.append(line[11:])
fingerprintFile.close()

In [116]:
##Converts the Lists of strings into list of Vectors
pfamVectorList=[]#List of vectors
for pfamString in pfamList:
    pfamVectorList.append(convertStringIntoVector(pfamString))
fingerprintVectorList=[]##List of vectors
for fingerprintString in fingerprintList:
    fingerprintVectorList.append(convertStringIntoVector(fingerprintString))

In [117]:
##Sets parameters for the training part of the formula (lambda*Identity*GramMatrix) and inverses it
dimensionGeneralMatrix= len (pfamVectorList)
lambdaa=0.01 #lambda seems to be a special character in python so lets put a double aa
generalGramMatrix = np.zeros((dimensionGeneralMatrix,dimensionGeneralMatrix))
for i in range (0, dimensionGeneralMatrix):
    for j in range (0, dimensionGeneralMatrix):
        generalGramMatrix [i,j]=gaussianKernelForPfam(pfamVectorList[i],pfamVectorList[j])#Kx with upper case

In [118]:
sampleOf206Numbers=listOfRandomNumbers[0:206]
sampleOf825Numbers= listOfRandomNumbers[206:]
####This is just the same script repeated several times for the 5 subsets.
####The last time, the length of the training and the test set changes, but the rest is the same

##Creates the training part of the formula once you know the specific clusters in the training set

dimensionTrainingSet = 825 #We divide 1031 by 5 for crossvalidation. We have 825 in the training and 206 in the test
dimensionTestSet = 206
identityMatrix=np.identity(dimensionTrainingSet)##Part of the formula
specificGramMatrix = np.zeros((dimensionTrainingSet,dimensionTrainingSet))
for i in range (0, dimensionTrainingSet):
    for j in range (0, dimensionTrainingSet):
        iIndex=sampleOf825Numbers[i]
        jIndex=sampleOf825Numbers[j]
        specificGramMatrix[i,j]=generalGramMatrix[iIndex,jIndex]
inverseMatrix = np.linalg.inv( ( lambdaa * identityMatrix ) + specificGramMatrix )##Part of the formula

In [119]:
def calculatingIOKRScore (xVector, yVector):
    ky=np.zeros((dimensionTrainingSet,1))##kx lower case
    kx=np.zeros((dimensionTrainingSet,1))##ky lower case
    for i in range (0,dimensionTrainingSet):
        iIndex=sampleOf825Numbers[i]
        kx[i]=gaussianKernelForPfam(pfamVectorList[iIndex],xVector)
        ky[i]=gaussianKernelForFingerprint(fingerprintVectorList[iIndex],yVector)
    kyt=np.transpose(ky)##Transposing ky
    result= np.matmul(np.matmul(kyt,inverseMatrix), kx)##Returns a number when doing the multiplication of the matrixes
    return result

In [120]:
score = np.zeros((206, 206))
for i in range (0, 206):
    for j in range (0, 206):
        iIndex=sampleOf206Numbers[i]
        jIndex=sampleOf206Numbers[j]
        score[i,j]= calculatingIOKRScore(pfamVectorList[iIndex],fingerprintVectorList[jIndex])
np.save("score16",score)

In [121]:
sampleOf206Numbers=listOfRandomNumbers[206:412]
sampleOf825Numbers= listOfRandomNumbers[0:206]+listOfRandomNumbers[412:]
##Creates the training part of the formula once you know the specific clusters in the training
dimensionTrainingSet = 825 #We divide 1031 by 5 for crossvalidation. We have 825 in the rest
dimensionTestSet = 206
identityMatrix=np.identity(dimensionTrainingSet)##Part of the formula
specificGramMatrix = np.zeros((dimensionTrainingSet,dimensionTrainingSet))
for i in range (0, dimensionTrainingSet):
    for j in range (0, dimensionTrainingSet):
        iIndex=sampleOf825Numbers[i]
        jIndex=sampleOf825Numbers[j]
        specificGramMatrix[i,j]=generalGramMatrix[iIndex,jIndex]
inverseMatrix = np.linalg.inv( ( lambdaa * identityMatrix ) + specificGramMatrix )##Part of the formula

In [122]:
def calculatingIOKRScore (xVector, yVector):
    ky=np.zeros((dimensionTrainingSet,1))##kx lower case
    kx=np.zeros((dimensionTrainingSet,1))##ky lower case
    for i in range (0,dimensionTrainingSet):
        iIndex=sampleOf825Numbers[i]
        kx[i]=gaussianKernelForPfam(pfamVectorList[iIndex],xVector)
        ky[i]=gaussianKernelForFingerprint(fingerprintVectorList[iIndex],yVector)
    kyt=np.transpose(ky)##Transposing ky
    result= np.matmul(np.matmul(kyt,inverseMatrix), kx)##Returns a number when doing the multiplication of the matrixes
    return result

In [123]:
score2 = np.zeros((206, 206))
for i in range (0, 206):
    for j in range (0, 206):
        iIndex=sampleOf206Numbers[i]
        jIndex=sampleOf206Numbers[j]
        score2[i,j]= calculatingIOKRScore(pfamVectorList[iIndex],fingerprintVectorList[jIndex])
np.save("score17",score2)

In [124]:
sampleOf206Numbers=listOfRandomNumbers[412:618]
sampleOf825Numbers= listOfRandomNumbers[0:412]+listOfRandomNumbers[618:]
##Creates the training part of the formula once you know the specific clusters in the training
dimensionTrainingSet = 825 #We divide 1031 by 5 for crossvalidation. We have 825 in the rest
dimensionTestSet = 206
identityMatrix=np.identity(dimensionTrainingSet)##Part of the formula
specificGramMatrix = np.zeros((dimensionTrainingSet,dimensionTrainingSet))
for i in range (0, dimensionTrainingSet):
    for j in range (0, dimensionTrainingSet):
        iIndex=sampleOf825Numbers[i]
        jIndex=sampleOf825Numbers[j]
        specificGramMatrix[i,j]=generalGramMatrix[iIndex,jIndex]
inverseMatrix = np.linalg.inv( ( lambdaa * identityMatrix ) + specificGramMatrix )##Part of the formula

In [125]:
def calculatingIOKRScore (xVector, yVector):
    ky=np.zeros((dimensionTrainingSet,1))##kx lower case
    kx=np.zeros((dimensionTrainingSet,1))##ky lower case
    for i in range (0,dimensionTrainingSet):
        iIndex=sampleOf825Numbers[i]
        kx[i]=gaussianKernelForPfam(pfamVectorList[iIndex],xVector)
        ky[i]=gaussianKernelForFingerprint(fingerprintVectorList[iIndex],yVector)
    kyt=np.transpose(ky)##Transposing ky
    result= np.matmul(np.matmul(kyt,inverseMatrix), kx)##Returns a number when doing the multiplication of the matrixes
    return result

In [126]:
score3 = np.zeros((206, 206))
for i in range (0, 206):
    for j in range (0, 206):
        iIndex=sampleOf206Numbers[i]
        jIndex=sampleOf206Numbers[j]
        score3[i,j]= calculatingIOKRScore(pfamVectorList[iIndex],fingerprintVectorList[jIndex])
np.save("score18",score3)

In [127]:
sampleOf206Numbers=listOfRandomNumbers[618:824]
sampleOf825Numbers= listOfRandomNumbers[0:618]+listOfRandomNumbers[824:]
##Creates the training part of the formula once you know the specific clusters in the training
dimensionTrainingSet = 825 #We divide 1031 by 5 for crossvalidation. We have 825 in the rest
dimensionTestSet = 206
identityMatrix=np.identity(dimensionTrainingSet)##Part of the formula
specificGramMatrix = np.zeros((dimensionTrainingSet,dimensionTrainingSet))
for i in range (0, dimensionTrainingSet):
    for j in range (0, dimensionTrainingSet):
        iIndex=sampleOf825Numbers[i]
        jIndex=sampleOf825Numbers[j]
        specificGramMatrix[i,j]=generalGramMatrix[iIndex,jIndex]
inverseMatrix = np.linalg.inv( ( lambdaa * identityMatrix ) + specificGramMatrix )##Part of the formula

In [128]:
def calculatingIOKRScore (xVector, yVector):
    ky=np.zeros((dimensionTrainingSet,1))##kx lower case
    kx=np.zeros((dimensionTrainingSet,1))##ky lower case
    for i in range (0,dimensionTrainingSet):
        iIndex=sampleOf825Numbers[i]
        kx[i]=gaussianKernelForPfam(pfamVectorList[iIndex],xVector)
        ky[i]=gaussianKernelForFingerprint(fingerprintVectorList[iIndex],yVector)
    kyt=np.transpose(ky)##Transposing ky
    result= np.matmul(np.matmul(kyt,inverseMatrix), kx)##Returns a number when doing the multiplication of the matrixes
    return result

In [129]:
score4 = np.zeros((206, 206))
for i in range (0, 206):
    for j in range (0, 206):
        iIndex=sampleOf206Numbers[i]
        jIndex=sampleOf206Numbers[j]
        score4[i,j]= calculatingIOKRScore(pfamVectorList[iIndex],fingerprintVectorList[jIndex])
np.save("score19",score4)

In [130]:
sampleOf206Numbers=listOfRandomNumbers[824:]
sampleOf825Numbers= listOfRandomNumbers[0:824]
##Creates the training part of the formula once you know the specific clusters in the training
dimensionTrainingSet = 824 #We divide 1031 by 5 for crossvalidation. We have 825 in the rest
dimensionTestSet = 207
identityMatrix=np.identity(dimensionTrainingSet)##Part of the formula
specificGramMatrix = np.zeros((dimensionTrainingSet,dimensionTrainingSet))
for i in range (0, dimensionTrainingSet):
    for j in range (0, dimensionTrainingSet):
        iIndex=sampleOf825Numbers[i]
        jIndex=sampleOf825Numbers[j]
        specificGramMatrix[i,j]=generalGramMatrix[iIndex,jIndex]
inverseMatrix = np.linalg.inv( ( lambdaa * identityMatrix ) + specificGramMatrix )##Part of the formula

In [131]:
def calculatingIOKRScore (xVector, yVector):
    ky=np.zeros((dimensionTrainingSet,1))##kx lower case
    kx=np.zeros((dimensionTrainingSet,1))##ky lower case
    for i in range (0,dimensionTrainingSet):
        iIndex=sampleOf825Numbers[i]
        kx[i]=gaussianKernelForPfam(pfamVectorList[iIndex],xVector)
        ky[i]=gaussianKernelForFingerprint(fingerprintVectorList[iIndex],yVector)
    kyt=np.transpose(ky)##Transposing ky
    result= np.matmul(np.matmul(kyt,inverseMatrix), kx)##Returns a number when doing the multiplication of the matrixes
    return result

In [132]:
score5 = np.zeros((207, 207))
for i in range (0, 207):
    for j in range (0, 207):
        iIndex=sampleOf206Numbers[i]
        jIndex=sampleOf206Numbers[j]
        score5[i,j]= calculatingIOKRScore(pfamVectorList[iIndex],fingerprintVectorList[jIndex])
np.save("score20",score5)

In [133]:
####The cell with import random has been deleted as we want to use the same order for all the parameters
## GammaX = 0.003
## GammaY = 0.03
## Lambda = 0.01

In [134]:
cd C:\Users\Yoo\Desktop\summerProject\hmmer-3.2.1\trial\usefulFiles\machineLearningKernelPfamFingerPrints\IKOR_TryingDifferentParameters\sameOrderForAllConfigurations\firstExperiment

C:\Users\Yoo\Desktop\summerProject\hmmer-3.2.1\trial\usefulFiles\machineLearningKernelPfamFingerPrints\IKOR_TryingDifferentParameters\sameOrderForAllConfigurations\firstExperiment


In [135]:
import numpy as np

In [136]:
def convertStringIntoVector (String):##Receives a binary vector in the form of a String
    intoList=list(String)
    vector=np.asarray(intoList,dtype=int)
    return vector

In [137]:
def gaussianKernelForPfam (vector1, vector2):##Does the gaussian Kernel for two vectors
    gamma=0.003
    euclideanDistance=np.linalg.norm(vector1-vector2)
    kernelValue=np.exp(-gamma*(euclideanDistance**2))
    return kernelValue

In [138]:
def gaussianKernelForFingerprint (vector1, vector2):##We can have a different gamma in this kernel if we want
    gamma=0.03
    euclideanDistance=np.linalg.norm(vector1-vector2)
    kernelValue=np.exp(-gamma*(euclideanDistance**2))
    return kernelValue

In [139]:
##Opens the two files and creates one list of Strings for pfam and another one for fingerprints
pfamFile= open ("pfamDomains0and1ForAllClusters.txt")
pfamList=[]##List of Strings
for line in pfamFile:
    line=line.strip("\n")
    pfamList.append(line[11:])
pfamFile.close()
fingerprintFile=open ("fingerPrintsForIndividualClustersNOT478.txt")
fingerprintList=[]##List of strings
for line in fingerprintFile:
    line=line.strip("\n")
    fingerprintList.append(line[11:])
fingerprintFile.close()

In [140]:
##Converts the Lists of strings into list of Vectors
pfamVectorList=[]#List of vectors
for pfamString in pfamList:
    pfamVectorList.append(convertStringIntoVector(pfamString))
fingerprintVectorList=[]##List of vectors
for fingerprintString in fingerprintList:
    fingerprintVectorList.append(convertStringIntoVector(fingerprintString))

In [141]:
##Sets parameters for the training part of the formula (lambda*Identity*GramMatrix) and inverses it
dimensionGeneralMatrix= len (pfamVectorList)
lambdaa=0.01 #lambda seems to be a special character in python so lets put a double aa
generalGramMatrix = np.zeros((dimensionGeneralMatrix,dimensionGeneralMatrix))
for i in range (0, dimensionGeneralMatrix):
    for j in range (0, dimensionGeneralMatrix):
        generalGramMatrix [i,j]=gaussianKernelForPfam(pfamVectorList[i],pfamVectorList[j])#Kx with upper case

In [142]:
sampleOf206Numbers=listOfRandomNumbers[0:206]
sampleOf825Numbers= listOfRandomNumbers[206:]
####This is just the same script repeated several times for the 5 subsets.
####The last time, the length of the training and the test set changes, but the rest is the same

##Creates the training part of the formula once you know the specific clusters in the training set

dimensionTrainingSet = 825 #We divide 1031 by 5 for crossvalidation. We have 825 in the training and 206 in the test
dimensionTestSet = 206
identityMatrix=np.identity(dimensionTrainingSet)##Part of the formula
specificGramMatrix = np.zeros((dimensionTrainingSet,dimensionTrainingSet))
for i in range (0, dimensionTrainingSet):
    for j in range (0, dimensionTrainingSet):
        iIndex=sampleOf825Numbers[i]
        jIndex=sampleOf825Numbers[j]
        specificGramMatrix[i,j]=generalGramMatrix[iIndex,jIndex]
inverseMatrix = np.linalg.inv( ( lambdaa * identityMatrix ) + specificGramMatrix )##Part of the formula

In [143]:
def calculatingIOKRScore (xVector, yVector):
    ky=np.zeros((dimensionTrainingSet,1))##kx lower case
    kx=np.zeros((dimensionTrainingSet,1))##ky lower case
    for i in range (0,dimensionTrainingSet):
        iIndex=sampleOf825Numbers[i]
        kx[i]=gaussianKernelForPfam(pfamVectorList[iIndex],xVector)
        ky[i]=gaussianKernelForFingerprint(fingerprintVectorList[iIndex],yVector)
    kyt=np.transpose(ky)##Transposing ky
    result= np.matmul(np.matmul(kyt,inverseMatrix), kx)##Returns a number when doing the multiplication of the matrixes
    return result

In [144]:
score = np.zeros((206, 206))
for i in range (0, 206):
    for j in range (0, 206):
        iIndex=sampleOf206Numbers[i]
        jIndex=sampleOf206Numbers[j]
        score[i,j]= calculatingIOKRScore(pfamVectorList[iIndex],fingerprintVectorList[jIndex])
np.save("score21",score)

In [145]:
sampleOf206Numbers=listOfRandomNumbers[206:412]
sampleOf825Numbers= listOfRandomNumbers[0:206]+listOfRandomNumbers[412:]
##Creates the training part of the formula once you know the specific clusters in the training
dimensionTrainingSet = 825 #We divide 1031 by 5 for crossvalidation. We have 825 in the rest
dimensionTestSet = 206
identityMatrix=np.identity(dimensionTrainingSet)##Part of the formula
specificGramMatrix = np.zeros((dimensionTrainingSet,dimensionTrainingSet))
for i in range (0, dimensionTrainingSet):
    for j in range (0, dimensionTrainingSet):
        iIndex=sampleOf825Numbers[i]
        jIndex=sampleOf825Numbers[j]
        specificGramMatrix[i,j]=generalGramMatrix[iIndex,jIndex]
inverseMatrix = np.linalg.inv( ( lambdaa * identityMatrix ) + specificGramMatrix )##Part of the formula

In [146]:
def calculatingIOKRScore (xVector, yVector):
    ky=np.zeros((dimensionTrainingSet,1))##kx lower case
    kx=np.zeros((dimensionTrainingSet,1))##ky lower case
    for i in range (0,dimensionTrainingSet):
        iIndex=sampleOf825Numbers[i]
        kx[i]=gaussianKernelForPfam(pfamVectorList[iIndex],xVector)
        ky[i]=gaussianKernelForFingerprint(fingerprintVectorList[iIndex],yVector)
    kyt=np.transpose(ky)##Transposing ky
    result= np.matmul(np.matmul(kyt,inverseMatrix), kx)##Returns a number when doing the multiplication of the matrixes
    return result

In [147]:
score2 = np.zeros((206, 206))
for i in range (0, 206):
    for j in range (0, 206):
        iIndex=sampleOf206Numbers[i]
        jIndex=sampleOf206Numbers[j]
        score2[i,j]= calculatingIOKRScore(pfamVectorList[iIndex],fingerprintVectorList[jIndex])
np.save("score22",score2)

In [148]:
sampleOf206Numbers=listOfRandomNumbers[412:618]
sampleOf825Numbers= listOfRandomNumbers[0:412]+listOfRandomNumbers[618:]
##Creates the training part of the formula once you know the specific clusters in the training
dimensionTrainingSet = 825 #We divide 1031 by 5 for crossvalidation. We have 825 in the rest
dimensionTestSet = 206
identityMatrix=np.identity(dimensionTrainingSet)##Part of the formula
specificGramMatrix = np.zeros((dimensionTrainingSet,dimensionTrainingSet))
for i in range (0, dimensionTrainingSet):
    for j in range (0, dimensionTrainingSet):
        iIndex=sampleOf825Numbers[i]
        jIndex=sampleOf825Numbers[j]
        specificGramMatrix[i,j]=generalGramMatrix[iIndex,jIndex]
inverseMatrix = np.linalg.inv( ( lambdaa * identityMatrix ) + specificGramMatrix )##Part of the formula

In [149]:
def calculatingIOKRScore (xVector, yVector):
    ky=np.zeros((dimensionTrainingSet,1))##kx lower case
    kx=np.zeros((dimensionTrainingSet,1))##ky lower case
    for i in range (0,dimensionTrainingSet):
        iIndex=sampleOf825Numbers[i]
        kx[i]=gaussianKernelForPfam(pfamVectorList[iIndex],xVector)
        ky[i]=gaussianKernelForFingerprint(fingerprintVectorList[iIndex],yVector)
    kyt=np.transpose(ky)##Transposing ky
    result= np.matmul(np.matmul(kyt,inverseMatrix), kx)##Returns a number when doing the multiplication of the matrixes
    return result

In [150]:
score3 = np.zeros((206, 206))
for i in range (0, 206):
    for j in range (0, 206):
        iIndex=sampleOf206Numbers[i]
        jIndex=sampleOf206Numbers[j]
        score3[i,j]= calculatingIOKRScore(pfamVectorList[iIndex],fingerprintVectorList[jIndex])
np.save("score23",score3)

In [151]:
sampleOf206Numbers=listOfRandomNumbers[618:824]
sampleOf825Numbers= listOfRandomNumbers[0:618]+listOfRandomNumbers[824:]
##Creates the training part of the formula once you know the specific clusters in the training
dimensionTrainingSet = 825 #We divide 1031 by 5 for crossvalidation. We have 825 in the rest
dimensionTestSet = 206
identityMatrix=np.identity(dimensionTrainingSet)##Part of the formula
specificGramMatrix = np.zeros((dimensionTrainingSet,dimensionTrainingSet))
for i in range (0, dimensionTrainingSet):
    for j in range (0, dimensionTrainingSet):
        iIndex=sampleOf825Numbers[i]
        jIndex=sampleOf825Numbers[j]
        specificGramMatrix[i,j]=generalGramMatrix[iIndex,jIndex]
inverseMatrix = np.linalg.inv( ( lambdaa * identityMatrix ) + specificGramMatrix )##Part of the formula

In [152]:
def calculatingIOKRScore (xVector, yVector):
    ky=np.zeros((dimensionTrainingSet,1))##kx lower case
    kx=np.zeros((dimensionTrainingSet,1))##ky lower case
    for i in range (0,dimensionTrainingSet):
        iIndex=sampleOf825Numbers[i]
        kx[i]=gaussianKernelForPfam(pfamVectorList[iIndex],xVector)
        ky[i]=gaussianKernelForFingerprint(fingerprintVectorList[iIndex],yVector)
    kyt=np.transpose(ky)##Transposing ky
    result= np.matmul(np.matmul(kyt,inverseMatrix), kx)##Returns a number when doing the multiplication of the matrixes
    return result

In [153]:
score4 = np.zeros((206, 206))
for i in range (0, 206):
    for j in range (0, 206):
        iIndex=sampleOf206Numbers[i]
        jIndex=sampleOf206Numbers[j]
        score4[i,j]= calculatingIOKRScore(pfamVectorList[iIndex],fingerprintVectorList[jIndex])
np.save("score24",score4)

In [154]:
sampleOf206Numbers=listOfRandomNumbers[824:]
sampleOf825Numbers= listOfRandomNumbers[0:824]
##Creates the training part of the formula once you know the specific clusters in the training
dimensionTrainingSet = 824 #We divide 1031 by 5 for crossvalidation. We have 825 in the rest
dimensionTestSet = 207
identityMatrix=np.identity(dimensionTrainingSet)##Part of the formula
specificGramMatrix = np.zeros((dimensionTrainingSet,dimensionTrainingSet))
for i in range (0, dimensionTrainingSet):
    for j in range (0, dimensionTrainingSet):
        iIndex=sampleOf825Numbers[i]
        jIndex=sampleOf825Numbers[j]
        specificGramMatrix[i,j]=generalGramMatrix[iIndex,jIndex]
inverseMatrix = np.linalg.inv( ( lambdaa * identityMatrix ) + specificGramMatrix )##Part of the formula

In [155]:
def calculatingIOKRScore (xVector, yVector):
    ky=np.zeros((dimensionTrainingSet,1))##kx lower case
    kx=np.zeros((dimensionTrainingSet,1))##ky lower case
    for i in range (0,dimensionTrainingSet):
        iIndex=sampleOf825Numbers[i]
        kx[i]=gaussianKernelForPfam(pfamVectorList[iIndex],xVector)
        ky[i]=gaussianKernelForFingerprint(fingerprintVectorList[iIndex],yVector)
    kyt=np.transpose(ky)##Transposing ky
    result= np.matmul(np.matmul(kyt,inverseMatrix), kx)##Returns a number when doing the multiplication of the matrixes
    return result

In [156]:
score5 = np.zeros((207, 207))
for i in range (0, 207):
    for j in range (0, 207):
        iIndex=sampleOf206Numbers[i]
        jIndex=sampleOf206Numbers[j]
        score5[i,j]= calculatingIOKRScore(pfamVectorList[iIndex],fingerprintVectorList[jIndex])
np.save("score25",score5)

In [157]:
####The cell with import random has been deleted as we want to use the same order for all the parameters
## GammaX = 0.003
## GammaY = 0.1
## Lambda = 0.01

In [158]:
cd C:\Users\Yoo\Desktop\summerProject\hmmer-3.2.1\trial\usefulFiles\machineLearningKernelPfamFingerPrints\IKOR_TryingDifferentParameters\sameOrderForAllConfigurations\firstExperiment

C:\Users\Yoo\Desktop\summerProject\hmmer-3.2.1\trial\usefulFiles\machineLearningKernelPfamFingerPrints\IKOR_TryingDifferentParameters\sameOrderForAllConfigurations\firstExperiment


In [159]:
import numpy as np

In [160]:
def convertStringIntoVector (String):##Receives a binary vector in the form of a String
    intoList=list(String)
    vector=np.asarray(intoList,dtype=int)
    return vector

In [161]:
def gaussianKernelForPfam (vector1, vector2):##Does the gaussian Kernel for two vectors
    gamma=0.003
    euclideanDistance=np.linalg.norm(vector1-vector2)
    kernelValue=np.exp(-gamma*(euclideanDistance**2))
    return kernelValue

In [162]:
def gaussianKernelForFingerprint (vector1, vector2):##We can have a different gamma in this kernel if we want
    gamma=0.1
    euclideanDistance=np.linalg.norm(vector1-vector2)
    kernelValue=np.exp(-gamma*(euclideanDistance**2))
    return kernelValue

In [163]:
##Opens the two files and creates one list of Strings for pfam and another one for fingerprints
pfamFile= open ("pfamDomains0and1ForAllClusters.txt")
pfamList=[]##List of Strings
for line in pfamFile:
    line=line.strip("\n")
    pfamList.append(line[11:])
pfamFile.close()
fingerprintFile=open ("fingerPrintsForIndividualClustersNOT478.txt")
fingerprintList=[]##List of strings
for line in fingerprintFile:
    line=line.strip("\n")
    fingerprintList.append(line[11:])
fingerprintFile.close()

In [164]:
##Converts the Lists of strings into list of Vectors
pfamVectorList=[]#List of vectors
for pfamString in pfamList:
    pfamVectorList.append(convertStringIntoVector(pfamString))
fingerprintVectorList=[]##List of vectors
for fingerprintString in fingerprintList:
    fingerprintVectorList.append(convertStringIntoVector(fingerprintString))

In [165]:
##Sets parameters for the training part of the formula (lambda*Identity*GramMatrix) and inverses it
dimensionGeneralMatrix= len (pfamVectorList)
lambdaa=0.01 #lambda seems to be a special character in python so lets put a double aa
generalGramMatrix = np.zeros((dimensionGeneralMatrix,dimensionGeneralMatrix))
for i in range (0, dimensionGeneralMatrix):
    for j in range (0, dimensionGeneralMatrix):
        generalGramMatrix [i,j]=gaussianKernelForPfam(pfamVectorList[i],pfamVectorList[j])#Kx with upper case

In [166]:
sampleOf206Numbers=listOfRandomNumbers[0:206]
sampleOf825Numbers= listOfRandomNumbers[206:]
####This is just the same script repeated several times for the 5 subsets.
####The last time, the length of the training and the test set changes, but the rest is the same

##Creates the training part of the formula once you know the specific clusters in the training set

dimensionTrainingSet = 825 #We divide 1031 by 5 for crossvalidation. We have 825 in the training and 206 in the test
dimensionTestSet = 206
identityMatrix=np.identity(dimensionTrainingSet)##Part of the formula
specificGramMatrix = np.zeros((dimensionTrainingSet,dimensionTrainingSet))
for i in range (0, dimensionTrainingSet):
    for j in range (0, dimensionTrainingSet):
        iIndex=sampleOf825Numbers[i]
        jIndex=sampleOf825Numbers[j]
        specificGramMatrix[i,j]=generalGramMatrix[iIndex,jIndex]
inverseMatrix = np.linalg.inv( ( lambdaa * identityMatrix ) + specificGramMatrix )##Part of the formula

In [167]:
def calculatingIOKRScore (xVector, yVector):
    ky=np.zeros((dimensionTrainingSet,1))##kx lower case
    kx=np.zeros((dimensionTrainingSet,1))##ky lower case
    for i in range (0,dimensionTrainingSet):
        iIndex=sampleOf825Numbers[i]
        kx[i]=gaussianKernelForPfam(pfamVectorList[iIndex],xVector)
        ky[i]=gaussianKernelForFingerprint(fingerprintVectorList[iIndex],yVector)
    kyt=np.transpose(ky)##Transposing ky
    result= np.matmul(np.matmul(kyt,inverseMatrix), kx)##Returns a number when doing the multiplication of the matrixes
    return result

In [168]:
score = np.zeros((206, 206))
for i in range (0, 206):
    for j in range (0, 206):
        iIndex=sampleOf206Numbers[i]
        jIndex=sampleOf206Numbers[j]
        score[i,j]= calculatingIOKRScore(pfamVectorList[iIndex],fingerprintVectorList[jIndex])
np.save("score26",score)

In [169]:
sampleOf206Numbers=listOfRandomNumbers[206:412]
sampleOf825Numbers= listOfRandomNumbers[0:206]+listOfRandomNumbers[412:]
##Creates the training part of the formula once you know the specific clusters in the training
dimensionTrainingSet = 825 #We divide 1031 by 5 for crossvalidation. We have 825 in the rest
dimensionTestSet = 206
identityMatrix=np.identity(dimensionTrainingSet)##Part of the formula
specificGramMatrix = np.zeros((dimensionTrainingSet,dimensionTrainingSet))
for i in range (0, dimensionTrainingSet):
    for j in range (0, dimensionTrainingSet):
        iIndex=sampleOf825Numbers[i]
        jIndex=sampleOf825Numbers[j]
        specificGramMatrix[i,j]=generalGramMatrix[iIndex,jIndex]
inverseMatrix = np.linalg.inv( ( lambdaa * identityMatrix ) + specificGramMatrix )##Part of the formula

In [170]:
def calculatingIOKRScore (xVector, yVector):
    ky=np.zeros((dimensionTrainingSet,1))##kx lower case
    kx=np.zeros((dimensionTrainingSet,1))##ky lower case
    for i in range (0,dimensionTrainingSet):
        iIndex=sampleOf825Numbers[i]
        kx[i]=gaussianKernelForPfam(pfamVectorList[iIndex],xVector)
        ky[i]=gaussianKernelForFingerprint(fingerprintVectorList[iIndex],yVector)
    kyt=np.transpose(ky)##Transposing ky
    result= np.matmul(np.matmul(kyt,inverseMatrix), kx)##Returns a number when doing the multiplication of the matrixes
    return result

In [171]:
score2 = np.zeros((206, 206))
for i in range (0, 206):
    for j in range (0, 206):
        iIndex=sampleOf206Numbers[i]
        jIndex=sampleOf206Numbers[j]
        score2[i,j]= calculatingIOKRScore(pfamVectorList[iIndex],fingerprintVectorList[jIndex])
np.save("score27",score2)

In [172]:
sampleOf206Numbers=listOfRandomNumbers[412:618]
sampleOf825Numbers= listOfRandomNumbers[0:412]+listOfRandomNumbers[618:]
##Creates the training part of the formula once you know the specific clusters in the training
dimensionTrainingSet = 825 #We divide 1031 by 5 for crossvalidation. We have 825 in the rest
dimensionTestSet = 206
identityMatrix=np.identity(dimensionTrainingSet)##Part of the formula
specificGramMatrix = np.zeros((dimensionTrainingSet,dimensionTrainingSet))
for i in range (0, dimensionTrainingSet):
    for j in range (0, dimensionTrainingSet):
        iIndex=sampleOf825Numbers[i]
        jIndex=sampleOf825Numbers[j]
        specificGramMatrix[i,j]=generalGramMatrix[iIndex,jIndex]
inverseMatrix = np.linalg.inv( ( lambdaa * identityMatrix ) + specificGramMatrix )##Part of the formula

In [173]:
def calculatingIOKRScore (xVector, yVector):
    ky=np.zeros((dimensionTrainingSet,1))##kx lower case
    kx=np.zeros((dimensionTrainingSet,1))##ky lower case
    for i in range (0,dimensionTrainingSet):
        iIndex=sampleOf825Numbers[i]
        kx[i]=gaussianKernelForPfam(pfamVectorList[iIndex],xVector)
        ky[i]=gaussianKernelForFingerprint(fingerprintVectorList[iIndex],yVector)
    kyt=np.transpose(ky)##Transposing ky
    result= np.matmul(np.matmul(kyt,inverseMatrix), kx)##Returns a number when doing the multiplication of the matrixes
    return result

In [174]:
score3 = np.zeros((206, 206))
for i in range (0, 206):
    for j in range (0, 206):
        iIndex=sampleOf206Numbers[i]
        jIndex=sampleOf206Numbers[j]
        score3[i,j]= calculatingIOKRScore(pfamVectorList[iIndex],fingerprintVectorList[jIndex])
np.save("score28",score3)

In [175]:
sampleOf206Numbers=listOfRandomNumbers[618:824]
sampleOf825Numbers= listOfRandomNumbers[0:618]+listOfRandomNumbers[824:]
##Creates the training part of the formula once you know the specific clusters in the training
dimensionTrainingSet = 825 #We divide 1031 by 5 for crossvalidation. We have 825 in the rest
dimensionTestSet = 206
identityMatrix=np.identity(dimensionTrainingSet)##Part of the formula
specificGramMatrix = np.zeros((dimensionTrainingSet,dimensionTrainingSet))
for i in range (0, dimensionTrainingSet):
    for j in range (0, dimensionTrainingSet):
        iIndex=sampleOf825Numbers[i]
        jIndex=sampleOf825Numbers[j]
        specificGramMatrix[i,j]=generalGramMatrix[iIndex,jIndex]
inverseMatrix = np.linalg.inv( ( lambdaa * identityMatrix ) + specificGramMatrix )##Part of the formula

In [176]:
def calculatingIOKRScore (xVector, yVector):
    ky=np.zeros((dimensionTrainingSet,1))##kx lower case
    kx=np.zeros((dimensionTrainingSet,1))##ky lower case
    for i in range (0,dimensionTrainingSet):
        iIndex=sampleOf825Numbers[i]
        kx[i]=gaussianKernelForPfam(pfamVectorList[iIndex],xVector)
        ky[i]=gaussianKernelForFingerprint(fingerprintVectorList[iIndex],yVector)
    kyt=np.transpose(ky)##Transposing ky
    result= np.matmul(np.matmul(kyt,inverseMatrix), kx)##Returns a number when doing the multiplication of the matrixes
    return result

In [177]:
score4 = np.zeros((206, 206))
for i in range (0, 206):
    for j in range (0, 206):
        iIndex=sampleOf206Numbers[i]
        jIndex=sampleOf206Numbers[j]
        score4[i,j]= calculatingIOKRScore(pfamVectorList[iIndex],fingerprintVectorList[jIndex])
np.save("score29",score4)

In [178]:
sampleOf206Numbers=listOfRandomNumbers[824:]
sampleOf825Numbers= listOfRandomNumbers[0:824]
##Creates the training part of the formula once you know the specific clusters in the training
dimensionTrainingSet = 824 #We divide 1031 by 5 for crossvalidation. We have 825 in the rest
dimensionTestSet = 207
identityMatrix=np.identity(dimensionTrainingSet)##Part of the formula
specificGramMatrix = np.zeros((dimensionTrainingSet,dimensionTrainingSet))
for i in range (0, dimensionTrainingSet):
    for j in range (0, dimensionTrainingSet):
        iIndex=sampleOf825Numbers[i]
        jIndex=sampleOf825Numbers[j]
        specificGramMatrix[i,j]=generalGramMatrix[iIndex,jIndex]
inverseMatrix = np.linalg.inv( ( lambdaa * identityMatrix ) + specificGramMatrix )##Part of the formula

In [179]:
def calculatingIOKRScore (xVector, yVector):
    ky=np.zeros((dimensionTrainingSet,1))##kx lower case
    kx=np.zeros((dimensionTrainingSet,1))##ky lower case
    for i in range (0,dimensionTrainingSet):
        iIndex=sampleOf825Numbers[i]
        kx[i]=gaussianKernelForPfam(pfamVectorList[iIndex],xVector)
        ky[i]=gaussianKernelForFingerprint(fingerprintVectorList[iIndex],yVector)
    kyt=np.transpose(ky)##Transposing ky
    result= np.matmul(np.matmul(kyt,inverseMatrix), kx)##Returns a number when doing the multiplication of the matrixes
    return result

In [180]:
score5 = np.zeros((207, 207))
for i in range (0, 207):
    for j in range (0, 207):
        iIndex=sampleOf206Numbers[i]
        jIndex=sampleOf206Numbers[j]
        score5[i,j]= calculatingIOKRScore(pfamVectorList[iIndex],fingerprintVectorList[jIndex])
np.save("score30",score5)

In [181]:
####The cell with import random has been deleted as we want to use the same order for all the parameters
## GammaX = 0.003
## GammaY = 0.3
## Lambda = 0.01

In [182]:
cd C:\Users\Yoo\Desktop\summerProject\hmmer-3.2.1\trial\usefulFiles\machineLearningKernelPfamFingerPrints\IKOR_TryingDifferentParameters\sameOrderForAllConfigurations\firstExperiment

C:\Users\Yoo\Desktop\summerProject\hmmer-3.2.1\trial\usefulFiles\machineLearningKernelPfamFingerPrints\IKOR_TryingDifferentParameters\sameOrderForAllConfigurations\firstExperiment


In [183]:
import numpy as np

In [184]:
def convertStringIntoVector (String):##Receives a binary vector in the form of a String
    intoList=list(String)
    vector=np.asarray(intoList,dtype=int)
    return vector

In [185]:
def gaussianKernelForPfam (vector1, vector2):##Does the gaussian Kernel for two vectors
    gamma=0.003
    euclideanDistance=np.linalg.norm(vector1-vector2)
    kernelValue=np.exp(-gamma*(euclideanDistance**2))
    return kernelValue

In [186]:
def gaussianKernelForFingerprint (vector1, vector2):##We can have a different gamma in this kernel if we want
    gamma=0.3
    euclideanDistance=np.linalg.norm(vector1-vector2)
    kernelValue=np.exp(-gamma*(euclideanDistance**2))
    return kernelValue

In [187]:
##Opens the two files and creates one list of Strings for pfam and another one for fingerprints
pfamFile= open ("pfamDomains0and1ForAllClusters.txt")
pfamList=[]##List of Strings
for line in pfamFile:
    line=line.strip("\n")
    pfamList.append(line[11:])
pfamFile.close()
fingerprintFile=open ("fingerPrintsForIndividualClustersNOT478.txt")
fingerprintList=[]##List of strings
for line in fingerprintFile:
    line=line.strip("\n")
    fingerprintList.append(line[11:])
fingerprintFile.close()

In [188]:
##Converts the Lists of strings into list of Vectors
pfamVectorList=[]#List of vectors
for pfamString in pfamList:
    pfamVectorList.append(convertStringIntoVector(pfamString))
fingerprintVectorList=[]##List of vectors
for fingerprintString in fingerprintList:
    fingerprintVectorList.append(convertStringIntoVector(fingerprintString))

In [189]:
##Sets parameters for the training part of the formula (lambda*Identity*GramMatrix) and inverses it
dimensionGeneralMatrix= len (pfamVectorList)
lambdaa=0.01 #lambda seems to be a special character in python so lets put a double aa
generalGramMatrix = np.zeros((dimensionGeneralMatrix,dimensionGeneralMatrix))
for i in range (0, dimensionGeneralMatrix):
    for j in range (0, dimensionGeneralMatrix):
        generalGramMatrix [i,j]=gaussianKernelForPfam(pfamVectorList[i],pfamVectorList[j])#Kx with upper case

In [190]:
sampleOf206Numbers=listOfRandomNumbers[0:206]
sampleOf825Numbers= listOfRandomNumbers[206:]
####This is just the same script repeated several times for the 5 subsets.
####The last time, the length of the training and the test set changes, but the rest is the same

##Creates the training part of the formula once you know the specific clusters in the training set

dimensionTrainingSet = 825 #We divide 1031 by 5 for crossvalidation. We have 825 in the training and 206 in the test
dimensionTestSet = 206
identityMatrix=np.identity(dimensionTrainingSet)##Part of the formula
specificGramMatrix = np.zeros((dimensionTrainingSet,dimensionTrainingSet))
for i in range (0, dimensionTrainingSet):
    for j in range (0, dimensionTrainingSet):
        iIndex=sampleOf825Numbers[i]
        jIndex=sampleOf825Numbers[j]
        specificGramMatrix[i,j]=generalGramMatrix[iIndex,jIndex]
inverseMatrix = np.linalg.inv( ( lambdaa * identityMatrix ) + specificGramMatrix )##Part of the formula

In [191]:
def calculatingIOKRScore (xVector, yVector):
    ky=np.zeros((dimensionTrainingSet,1))##kx lower case
    kx=np.zeros((dimensionTrainingSet,1))##ky lower case
    for i in range (0,dimensionTrainingSet):
        iIndex=sampleOf825Numbers[i]
        kx[i]=gaussianKernelForPfam(pfamVectorList[iIndex],xVector)
        ky[i]=gaussianKernelForFingerprint(fingerprintVectorList[iIndex],yVector)
    kyt=np.transpose(ky)##Transposing ky
    result= np.matmul(np.matmul(kyt,inverseMatrix), kx)##Returns a number when doing the multiplication of the matrixes
    return result

In [192]:
score = np.zeros((206, 206))
for i in range (0, 206):
    for j in range (0, 206):
        iIndex=sampleOf206Numbers[i]
        jIndex=sampleOf206Numbers[j]
        score[i,j]= calculatingIOKRScore(pfamVectorList[iIndex],fingerprintVectorList[jIndex])
np.save("score31",score)

In [193]:
sampleOf206Numbers=listOfRandomNumbers[206:412]
sampleOf825Numbers= listOfRandomNumbers[0:206]+listOfRandomNumbers[412:]
##Creates the training part of the formula once you know the specific clusters in the training
dimensionTrainingSet = 825 #We divide 1031 by 5 for crossvalidation. We have 825 in the rest
dimensionTestSet = 206
identityMatrix=np.identity(dimensionTrainingSet)##Part of the formula
specificGramMatrix = np.zeros((dimensionTrainingSet,dimensionTrainingSet))
for i in range (0, dimensionTrainingSet):
    for j in range (0, dimensionTrainingSet):
        iIndex=sampleOf825Numbers[i]
        jIndex=sampleOf825Numbers[j]
        specificGramMatrix[i,j]=generalGramMatrix[iIndex,jIndex]
inverseMatrix = np.linalg.inv( ( lambdaa * identityMatrix ) + specificGramMatrix )##Part of the formula

In [194]:
def calculatingIOKRScore (xVector, yVector):
    ky=np.zeros((dimensionTrainingSet,1))##kx lower case
    kx=np.zeros((dimensionTrainingSet,1))##ky lower case
    for i in range (0,dimensionTrainingSet):
        iIndex=sampleOf825Numbers[i]
        kx[i]=gaussianKernelForPfam(pfamVectorList[iIndex],xVector)
        ky[i]=gaussianKernelForFingerprint(fingerprintVectorList[iIndex],yVector)
    kyt=np.transpose(ky)##Transposing ky
    result= np.matmul(np.matmul(kyt,inverseMatrix), kx)##Returns a number when doing the multiplication of the matrixes
    return result

In [195]:
score2 = np.zeros((206, 206))
for i in range (0, 206):
    for j in range (0, 206):
        iIndex=sampleOf206Numbers[i]
        jIndex=sampleOf206Numbers[j]
        score2[i,j]= calculatingIOKRScore(pfamVectorList[iIndex],fingerprintVectorList[jIndex])
np.save("score32",score2)

In [196]:
sampleOf206Numbers=listOfRandomNumbers[412:618]
sampleOf825Numbers= listOfRandomNumbers[0:412]+listOfRandomNumbers[618:]
##Creates the training part of the formula once you know the specific clusters in the training
dimensionTrainingSet = 825 #We divide 1031 by 5 for crossvalidation. We have 825 in the rest
dimensionTestSet = 206
identityMatrix=np.identity(dimensionTrainingSet)##Part of the formula
specificGramMatrix = np.zeros((dimensionTrainingSet,dimensionTrainingSet))
for i in range (0, dimensionTrainingSet):
    for j in range (0, dimensionTrainingSet):
        iIndex=sampleOf825Numbers[i]
        jIndex=sampleOf825Numbers[j]
        specificGramMatrix[i,j]=generalGramMatrix[iIndex,jIndex]
inverseMatrix = np.linalg.inv( ( lambdaa * identityMatrix ) + specificGramMatrix )##Part of the formula

In [197]:
def calculatingIOKRScore (xVector, yVector):
    ky=np.zeros((dimensionTrainingSet,1))##kx lower case
    kx=np.zeros((dimensionTrainingSet,1))##ky lower case
    for i in range (0,dimensionTrainingSet):
        iIndex=sampleOf825Numbers[i]
        kx[i]=gaussianKernelForPfam(pfamVectorList[iIndex],xVector)
        ky[i]=gaussianKernelForFingerprint(fingerprintVectorList[iIndex],yVector)
    kyt=np.transpose(ky)##Transposing ky
    result= np.matmul(np.matmul(kyt,inverseMatrix), kx)##Returns a number when doing the multiplication of the matrixes
    return result

In [198]:
score3 = np.zeros((206, 206))
for i in range (0, 206):
    for j in range (0, 206):
        iIndex=sampleOf206Numbers[i]
        jIndex=sampleOf206Numbers[j]
        score3[i,j]= calculatingIOKRScore(pfamVectorList[iIndex],fingerprintVectorList[jIndex])
np.save("score33",score3)

In [199]:
sampleOf206Numbers=listOfRandomNumbers[618:824]
sampleOf825Numbers= listOfRandomNumbers[0:618]+listOfRandomNumbers[824:]
##Creates the training part of the formula once you know the specific clusters in the training
dimensionTrainingSet = 825 #We divide 1031 by 5 for crossvalidation. We have 825 in the rest
dimensionTestSet = 206
identityMatrix=np.identity(dimensionTrainingSet)##Part of the formula
specificGramMatrix = np.zeros((dimensionTrainingSet,dimensionTrainingSet))
for i in range (0, dimensionTrainingSet):
    for j in range (0, dimensionTrainingSet):
        iIndex=sampleOf825Numbers[i]
        jIndex=sampleOf825Numbers[j]
        specificGramMatrix[i,j]=generalGramMatrix[iIndex,jIndex]
inverseMatrix = np.linalg.inv( ( lambdaa * identityMatrix ) + specificGramMatrix )##Part of the formula

In [200]:
def calculatingIOKRScore (xVector, yVector):
    ky=np.zeros((dimensionTrainingSet,1))##kx lower case
    kx=np.zeros((dimensionTrainingSet,1))##ky lower case
    for i in range (0,dimensionTrainingSet):
        iIndex=sampleOf825Numbers[i]
        kx[i]=gaussianKernelForPfam(pfamVectorList[iIndex],xVector)
        ky[i]=gaussianKernelForFingerprint(fingerprintVectorList[iIndex],yVector)
    kyt=np.transpose(ky)##Transposing ky
    result= np.matmul(np.matmul(kyt,inverseMatrix), kx)##Returns a number when doing the multiplication of the matrixes
    return result

In [201]:
score4 = np.zeros((206, 206))
for i in range (0, 206):
    for j in range (0, 206):
        iIndex=sampleOf206Numbers[i]
        jIndex=sampleOf206Numbers[j]
        score4[i,j]= calculatingIOKRScore(pfamVectorList[iIndex],fingerprintVectorList[jIndex])
np.save("score34",score4)

In [202]:
sampleOf206Numbers=listOfRandomNumbers[824:]
sampleOf825Numbers= listOfRandomNumbers[0:824]
##Creates the training part of the formula once you know the specific clusters in the training
dimensionTrainingSet = 824 #We divide 1031 by 5 for crossvalidation. We have 825 in the rest
dimensionTestSet = 207
identityMatrix=np.identity(dimensionTrainingSet)##Part of the formula
specificGramMatrix = np.zeros((dimensionTrainingSet,dimensionTrainingSet))
for i in range (0, dimensionTrainingSet):
    for j in range (0, dimensionTrainingSet):
        iIndex=sampleOf825Numbers[i]
        jIndex=sampleOf825Numbers[j]
        specificGramMatrix[i,j]=generalGramMatrix[iIndex,jIndex]
inverseMatrix = np.linalg.inv( ( lambdaa * identityMatrix ) + specificGramMatrix )##Part of the formula

In [203]:
def calculatingIOKRScore (xVector, yVector):
    ky=np.zeros((dimensionTrainingSet,1))##kx lower case
    kx=np.zeros((dimensionTrainingSet,1))##ky lower case
    for i in range (0,dimensionTrainingSet):
        iIndex=sampleOf825Numbers[i]
        kx[i]=gaussianKernelForPfam(pfamVectorList[iIndex],xVector)
        ky[i]=gaussianKernelForFingerprint(fingerprintVectorList[iIndex],yVector)
    kyt=np.transpose(ky)##Transposing ky
    result= np.matmul(np.matmul(kyt,inverseMatrix), kx)##Returns a number when doing the multiplication of the matrixes
    return result

In [204]:
score5 = np.zeros((207, 207))
for i in range (0, 207):
    for j in range (0, 207):
        iIndex=sampleOf206Numbers[i]
        jIndex=sampleOf206Numbers[j]
        score5[i,j]= calculatingIOKRScore(pfamVectorList[iIndex],fingerprintVectorList[jIndex])
np.save("score35",score5)

In [205]:
####The cell with import random has been deleted as we want to use the same order for all the parameters
## GammaX = 3
## GammaY = 3
## Lambda = 0.01

In [206]:
cd C:\Users\Yoo\Desktop\summerProject\hmmer-3.2.1\trial\usefulFiles\machineLearningKernelPfamFingerPrints\IKOR_TryingDifferentParameters\sameOrderForAllConfigurations\firstExperiment

C:\Users\Yoo\Desktop\summerProject\hmmer-3.2.1\trial\usefulFiles\machineLearningKernelPfamFingerPrints\IKOR_TryingDifferentParameters\sameOrderForAllConfigurations\firstExperiment


In [207]:
import numpy as np

In [208]:
def convertStringIntoVector (String):##Receives a binary vector in the form of a String
    intoList=list(String)
    vector=np.asarray(intoList,dtype=int)
    return vector

In [209]:
def gaussianKernelForPfam (vector1, vector2):##Does the gaussian Kernel for two vectors
    gamma=3
    euclideanDistance=np.linalg.norm(vector1-vector2)
    kernelValue=np.exp(-gamma*(euclideanDistance**2))
    return kernelValue

In [210]:
def gaussianKernelForFingerprint (vector1, vector2):##We can have a different gamma in this kernel if we want
    gamma=3
    euclideanDistance=np.linalg.norm(vector1-vector2)
    kernelValue=np.exp(-gamma*(euclideanDistance**2))
    return kernelValue

In [211]:
##Opens the two files and creates one list of Strings for pfam and another one for fingerprints
pfamFile= open ("pfamDomains0and1ForAllClusters.txt")
pfamList=[]##List of Strings
for line in pfamFile:
    line=line.strip("\n")
    pfamList.append(line[11:])
pfamFile.close()
fingerprintFile=open ("fingerPrintsForIndividualClustersNOT478.txt")
fingerprintList=[]##List of strings
for line in fingerprintFile:
    line=line.strip("\n")
    fingerprintList.append(line[11:])
fingerprintFile.close()

In [212]:
##Converts the Lists of strings into list of Vectors
pfamVectorList=[]#List of vectors
for pfamString in pfamList:
    pfamVectorList.append(convertStringIntoVector(pfamString))
fingerprintVectorList=[]##List of vectors
for fingerprintString in fingerprintList:
    fingerprintVectorList.append(convertStringIntoVector(fingerprintString))

In [213]:
##Sets parameters for the training part of the formula (lambda*Identity*GramMatrix) and inverses it
dimensionGeneralMatrix= len (pfamVectorList)
lambdaa=0.01 #lambda seems to be a special character in python so lets put a double aa
generalGramMatrix = np.zeros((dimensionGeneralMatrix,dimensionGeneralMatrix))
for i in range (0, dimensionGeneralMatrix):
    for j in range (0, dimensionGeneralMatrix):
        generalGramMatrix [i,j]=gaussianKernelForPfam(pfamVectorList[i],pfamVectorList[j])#Kx with upper case

In [214]:
sampleOf206Numbers=listOfRandomNumbers[0:206]
sampleOf825Numbers= listOfRandomNumbers[206:]
####This is just the same script repeated several times for the 5 subsets.
####The last time, the length of the training and the test set changes, but the rest is the same

##Creates the training part of the formula once you know the specific clusters in the training set

dimensionTrainingSet = 825 #We divide 1031 by 5 for crossvalidation. We have 825 in the training and 206 in the test
dimensionTestSet = 206
identityMatrix=np.identity(dimensionTrainingSet)##Part of the formula
specificGramMatrix = np.zeros((dimensionTrainingSet,dimensionTrainingSet))
for i in range (0, dimensionTrainingSet):
    for j in range (0, dimensionTrainingSet):
        iIndex=sampleOf825Numbers[i]
        jIndex=sampleOf825Numbers[j]
        specificGramMatrix[i,j]=generalGramMatrix[iIndex,jIndex]
inverseMatrix = np.linalg.inv( ( lambdaa * identityMatrix ) + specificGramMatrix )##Part of the formula

In [215]:
def calculatingIOKRScore (xVector, yVector):
    ky=np.zeros((dimensionTrainingSet,1))##kx lower case
    kx=np.zeros((dimensionTrainingSet,1))##ky lower case
    for i in range (0,dimensionTrainingSet):
        iIndex=sampleOf825Numbers[i]
        kx[i]=gaussianKernelForPfam(pfamVectorList[iIndex],xVector)
        ky[i]=gaussianKernelForFingerprint(fingerprintVectorList[iIndex],yVector)
    kyt=np.transpose(ky)##Transposing ky
    result= np.matmul(np.matmul(kyt,inverseMatrix), kx)##Returns a number when doing the multiplication of the matrixes
    return result

In [216]:
score = np.zeros((206, 206))
for i in range (0, 206):
    for j in range (0, 206):
        iIndex=sampleOf206Numbers[i]
        jIndex=sampleOf206Numbers[j]
        score[i,j]= calculatingIOKRScore(pfamVectorList[iIndex],fingerprintVectorList[jIndex])
np.save("score36",score)

In [217]:
sampleOf206Numbers=listOfRandomNumbers[206:412]
sampleOf825Numbers= listOfRandomNumbers[0:206]+listOfRandomNumbers[412:]
##Creates the training part of the formula once you know the specific clusters in the training
dimensionTrainingSet = 825 #We divide 1031 by 5 for crossvalidation. We have 825 in the rest
dimensionTestSet = 206
identityMatrix=np.identity(dimensionTrainingSet)##Part of the formula
specificGramMatrix = np.zeros((dimensionTrainingSet,dimensionTrainingSet))
for i in range (0, dimensionTrainingSet):
    for j in range (0, dimensionTrainingSet):
        iIndex=sampleOf825Numbers[i]
        jIndex=sampleOf825Numbers[j]
        specificGramMatrix[i,j]=generalGramMatrix[iIndex,jIndex]
inverseMatrix = np.linalg.inv( ( lambdaa * identityMatrix ) + specificGramMatrix )##Part of the formula

In [218]:
def calculatingIOKRScore (xVector, yVector):
    ky=np.zeros((dimensionTrainingSet,1))##kx lower case
    kx=np.zeros((dimensionTrainingSet,1))##ky lower case
    for i in range (0,dimensionTrainingSet):
        iIndex=sampleOf825Numbers[i]
        kx[i]=gaussianKernelForPfam(pfamVectorList[iIndex],xVector)
        ky[i]=gaussianKernelForFingerprint(fingerprintVectorList[iIndex],yVector)
    kyt=np.transpose(ky)##Transposing ky
    result= np.matmul(np.matmul(kyt,inverseMatrix), kx)##Returns a number when doing the multiplication of the matrixes
    return result

In [219]:
score2 = np.zeros((206, 206))
for i in range (0, 206):
    for j in range (0, 206):
        iIndex=sampleOf206Numbers[i]
        jIndex=sampleOf206Numbers[j]
        score2[i,j]= calculatingIOKRScore(pfamVectorList[iIndex],fingerprintVectorList[jIndex])
np.save("score37",score2)

In [220]:
sampleOf206Numbers=listOfRandomNumbers[412:618]
sampleOf825Numbers= listOfRandomNumbers[0:412]+listOfRandomNumbers[618:]
##Creates the training part of the formula once you know the specific clusters in the training
dimensionTrainingSet = 825 #We divide 1031 by 5 for crossvalidation. We have 825 in the rest
dimensionTestSet = 206
identityMatrix=np.identity(dimensionTrainingSet)##Part of the formula
specificGramMatrix = np.zeros((dimensionTrainingSet,dimensionTrainingSet))
for i in range (0, dimensionTrainingSet):
    for j in range (0, dimensionTrainingSet):
        iIndex=sampleOf825Numbers[i]
        jIndex=sampleOf825Numbers[j]
        specificGramMatrix[i,j]=generalGramMatrix[iIndex,jIndex]
inverseMatrix = np.linalg.inv( ( lambdaa * identityMatrix ) + specificGramMatrix )##Part of the formula

In [221]:
def calculatingIOKRScore (xVector, yVector):
    ky=np.zeros((dimensionTrainingSet,1))##kx lower case
    kx=np.zeros((dimensionTrainingSet,1))##ky lower case
    for i in range (0,dimensionTrainingSet):
        iIndex=sampleOf825Numbers[i]
        kx[i]=gaussianKernelForPfam(pfamVectorList[iIndex],xVector)
        ky[i]=gaussianKernelForFingerprint(fingerprintVectorList[iIndex],yVector)
    kyt=np.transpose(ky)##Transposing ky
    result= np.matmul(np.matmul(kyt,inverseMatrix), kx)##Returns a number when doing the multiplication of the matrixes
    return result

In [222]:
score3 = np.zeros((206, 206))
for i in range (0, 206):
    for j in range (0, 206):
        iIndex=sampleOf206Numbers[i]
        jIndex=sampleOf206Numbers[j]
        score3[i,j]= calculatingIOKRScore(pfamVectorList[iIndex],fingerprintVectorList[jIndex])
np.save("score38",score3)

In [223]:
sampleOf206Numbers=listOfRandomNumbers[618:824]
sampleOf825Numbers= listOfRandomNumbers[0:618]+listOfRandomNumbers[824:]
##Creates the training part of the formula once you know the specific clusters in the training
dimensionTrainingSet = 825 #We divide 1031 by 5 for crossvalidation. We have 825 in the rest
dimensionTestSet = 206
identityMatrix=np.identity(dimensionTrainingSet)##Part of the formula
specificGramMatrix = np.zeros((dimensionTrainingSet,dimensionTrainingSet))
for i in range (0, dimensionTrainingSet):
    for j in range (0, dimensionTrainingSet):
        iIndex=sampleOf825Numbers[i]
        jIndex=sampleOf825Numbers[j]
        specificGramMatrix[i,j]=generalGramMatrix[iIndex,jIndex]
inverseMatrix = np.linalg.inv( ( lambdaa * identityMatrix ) + specificGramMatrix )##Part of the formula

In [224]:
def calculatingIOKRScore (xVector, yVector):
    ky=np.zeros((dimensionTrainingSet,1))##kx lower case
    kx=np.zeros((dimensionTrainingSet,1))##ky lower case
    for i in range (0,dimensionTrainingSet):
        iIndex=sampleOf825Numbers[i]
        kx[i]=gaussianKernelForPfam(pfamVectorList[iIndex],xVector)
        ky[i]=gaussianKernelForFingerprint(fingerprintVectorList[iIndex],yVector)
    kyt=np.transpose(ky)##Transposing ky
    result= np.matmul(np.matmul(kyt,inverseMatrix), kx)##Returns a number when doing the multiplication of the matrixes
    return result

In [225]:
score4 = np.zeros((206, 206))
for i in range (0, 206):
    for j in range (0, 206):
        iIndex=sampleOf206Numbers[i]
        jIndex=sampleOf206Numbers[j]
        score4[i,j]= calculatingIOKRScore(pfamVectorList[iIndex],fingerprintVectorList[jIndex])
np.save("score39",score4)

In [226]:
sampleOf206Numbers=listOfRandomNumbers[824:]
sampleOf825Numbers= listOfRandomNumbers[0:824]
##Creates the training part of the formula once you know the specific clusters in the training
dimensionTrainingSet = 824 #We divide 1031 by 5 for crossvalidation. We have 825 in the rest
dimensionTestSet = 207
identityMatrix=np.identity(dimensionTrainingSet)##Part of the formula
specificGramMatrix = np.zeros((dimensionTrainingSet,dimensionTrainingSet))
for i in range (0, dimensionTrainingSet):
    for j in range (0, dimensionTrainingSet):
        iIndex=sampleOf825Numbers[i]
        jIndex=sampleOf825Numbers[j]
        specificGramMatrix[i,j]=generalGramMatrix[iIndex,jIndex]
inverseMatrix = np.linalg.inv( ( lambdaa * identityMatrix ) + specificGramMatrix )##Part of the formula

In [227]:
def calculatingIOKRScore (xVector, yVector):
    ky=np.zeros((dimensionTrainingSet,1))##kx lower case
    kx=np.zeros((dimensionTrainingSet,1))##ky lower case
    for i in range (0,dimensionTrainingSet):
        iIndex=sampleOf825Numbers[i]
        kx[i]=gaussianKernelForPfam(pfamVectorList[iIndex],xVector)
        ky[i]=gaussianKernelForFingerprint(fingerprintVectorList[iIndex],yVector)
    kyt=np.transpose(ky)##Transposing ky
    result= np.matmul(np.matmul(kyt,inverseMatrix), kx)##Returns a number when doing the multiplication of the matrixes
    return result

In [228]:
score5 = np.zeros((207, 207))
for i in range (0, 207):
    for j in range (0, 207):
        iIndex=sampleOf206Numbers[i]
        jIndex=sampleOf206Numbers[j]
        score5[i,j]= calculatingIOKRScore(pfamVectorList[iIndex],fingerprintVectorList[jIndex])
np.save("score40",score5)